In [1]:
import numpy as np
import time

## MATplotlib
import matplotlib.pyplot as plt

## MDA
import MDAnalysis as mda
from MDAnalysis.analysis.leaflet import LeafletFinder
from MDAnalysis.analysis.distances import distance_array


# ## SKLearn
from sklearn.neighbors import NearestNeighbors

## Open3D
import open3d as o3d

## optimesh
import optimesh

import math as math


In [2]:
def fill_in_defect_matrix(gl2_leaflet, grid_coords, normals, box, depth_max, upper=False, tinygridradius=10):
    m_polar = 1.0
    m_aliphatic = 0.001
    dist_far = 0.8
    dist_clo = 0.6
    ## Initalize matrix with size of number of cells
    M_1 = np.ones(np.shape(grid_coords)[0])
    M_1[:] = np.nan
    upperval = 1
    if (upper == True):
        upperval = -1
        ## v2: added this: (v1, without this part belw.)
        # normals = normals * upperval
    
    ## Loop over all lipids in the leaflet
    for gl2 in gl2_leaflet[:]:
        res = gl2.residue # Get the residue
        gl2_coords = gl2.position # position of the GL2 in A
        res_coords = res.atoms.positions # all positions of all beads in the molecule in A
        
        ## Get index of the closest gridcell to the gl2_coordinate
        index_closest_cell = np.argmin((grid_coords[:,0] - gl2_coords[0])**2 + (grid_coords[:,1] - gl2_coords[1])**2 + (grid_coords[:,2] - gl2_coords[2])**2, axis=0 )
        
        ## Get the corresponding coordinates for the grid cell that is closest to the gl2 coordinate
        cell_coord = grid_coords[index_closest_cell] ## corresponding coordinate
        
        ## get averaged normal vector over small grid of radius 10A 
        normal = np.mean(normals[np.where(np.sum((grid_coords - gl2_coords)**2, axis=1) < tinygridradius**2)], axis=0)
        if np.isnan(normal[0]):
            print(normal)
            # print(gl2_coords)
            # print(np.min(np.sum((grid_coords - gl2_coords)**2, axis=1)))
            # print(np.where(np.sum((grid_coords - gl2_coords)**2, axis=1) < tinygridradius**2))
            pass
        else:
            
            ## Take an average of normal vectors
            avnormal = upperval * normal
            avnormal = avnormal / np.sqrt(np.sum(avnormal**2)) ## Normalize
        
            ## Move grid such that is goes through the GL2 bead of the lipid
            grid_coords_m = grid_coords + (gl2_coords[2]-cell_coord[2])*avnormal
            if (abs(grid_coords_m[index_closest_cell][2]-gl2_coords[2]) >0.2):
                print(grid_coords_m[index_closest_cell], gl2_coords)
        
            ## Move the grid such that is is 1.0 lower than the GL2.
            grid_coords_m = grid_coords_m + depth_max*avnormal ## Upper
            cell_coord = grid_coords_m[index_closest_cell]
        
            ## Get closest index of moved grid:
            index_closest_cell2 = np.argmin((grid_coords_m[:,0] - gl2_coords[0])**2 + (grid_coords_m[:,1] - gl2_coords[1])**2 + (grid_coords_m[:,2] - gl2_coords[2])**2, axis=0 )
            cell_coord2 = grid_coords_m[index_closest_cell2] ## corresponding cell
        
            ## Move positions of beads in molecule such that cell_coords2 = [0, 0, 0], origin
            dr_atom_ref = min_image_dist(res_coords, cell_coord2, box)
        
            ## Flip normal (currently pointed into the membrane) after flipping point out!
            normal_gl2 = normal ##[index_closest_cell]
            # normal_gl2 = -1 * np.mean(normals, axis=0)
        
            ## Calculate the inner product. If the inner product is below zero >> The bead is below the grid 1.0 below gl2
            cos_alpha = (np.inner(dr_atom_ref, normal_gl2) /
                              (np.linalg.norm(dr_atom_ref, axis=1) * 
                               np.linalg.norm(normal_gl2))) # division by norm not necessary, 
                                                            # sign of inner product enough to check depth criterion
            
            for a, atom in enumerate(res.atoms):  #[np.where(cos_alpha >= 0)]:
                atom_name = atom.name ## get atom name to determine whether it is aliphatic or not.
            
                atom_radius, m_value = get_radius_and_defect_type(atom_name, cos_alpha[a])
            
                dist_lim = (dist_clo+atom_radius)**2
                dist_meet = ((dist_far)+atom_radius)**2
            
                ## Move grid to location of lipid:
#             dist_lim = ((np.sqrt(3)/2)+atom_radius)**2
#             dist_meet = (1+atom_radius)**2

                ix_closest = np.argmin((grid_coords_m[:,0] - atom.position[0])**2 + (grid_coords_m[:,1] - atom.position[1])**2 + (grid_coords_m[:,2] - atom.position[2])**2, axis=0 )
                dist = np.sqrt(np.sum((atom.position-grid_coords_m[ix_closest])**2))
                minorplus = np.linalg.norm(atom.position) - np.linalg.norm(grid_coords_m[ix_closest])
                dist = (minorplus / abs(minorplus)) * dist
            
                grid_coords_mm = grid_coords_m + (dist)*avnormal
            
#             if (np.sqrt(np.sum((atom.position-grid_coords_mm[ix_closest])**2))>0.2):
#                 print(atom.position)
#                 print(grid_coords_mm[ix_closest])
#                 print(np.sqrt(np.sum((atom.position-grid_coords_mm[ix_closest])**2)))
            
                delta_x = np.sum((grid_coords_mm-atom.position)**2, axis=1)

                ix_lim = np.where(delta_x <= dist_lim)[0]
                ix_meet = np.where(delta_x <= dist_meet)[0]
            
                ix_meet = np.unique(ix_meet)
                for x1 in ix_lim:
                    if np.isnan(M_1[x1]):
                        if m_value == m_polar:
                            M_1[x1] = 1
                        else:   
                            M_1[x1] = 0
                
                M_1[ix_meet] += m_value
        
        
    return M_1


def fill_in_defect_matrix2(gl2_leaflet, grid_coords, normals, box, depth_max, upper=False, tinygridradius=10, cutoff=0.6):
    m_polar = 1.0
    m_aliphatic = 0.001
    dist_far = cutoff
    dist_clo = cutoff
    ## Initalize matrix with size of number of cells
    M_1 = np.ones(np.shape(grid_coords)[0])
    M_1[:] = np.nan
    upperval = 1
    if (upper == True):
        upperval = -1
        ## v2: added this: (v1, without this part belw.)
        # normals = normals * upperval
    
    ## Loop over all lipids in the leaflet
    for gl2 in gl2_leaflet[:]:
        res = gl2.residue # Get the residue
        gl2_coords = gl2.position # position of the GL2 in A
        res_coords = res.atoms.positions # all positions of all beads in the molecule in A
        
        ## Get index of the closest gridcell to the gl2_coordinate
        index_closest_cell = np.argmin((grid_coords[:,0] - gl2_coords[0])**2 + (grid_coords[:,1] - gl2_coords[1])**2 + (grid_coords[:,2] - gl2_coords[2])**2, axis=0 )
        
        ## Get the corresponding coordinates for the grid cell that is closest to the gl2 coordinate
        cell_coord = grid_coords[index_closest_cell] ## corresponding coordinate
        
        ## get averaged normal vector over small grid of radius 10A 
        normal = np.mean(normals[np.where(np.sum((grid_coords - gl2_coords)**2, axis=1) < tinygridradius**2)], axis=0)
        if np.isnan(normal[0]):
            print(normal)
            # print(gl2_coords)
            # print(np.min(np.sum((grid_coords - gl2_coords)**2, axis=1)))
            # print(np.where(np.sum((grid_coords - gl2_coords)**2, axis=1) < tinygridradius**2))
            pass
        else:
            
            ## Take an average of normal vectors
            avnormal = upperval * normal
            avnormal = avnormal / np.sqrt(np.sum(avnormal**2)) ## Normalize
        
            ## Move grid such that is goes through the GL2 bead of the lipid
            grid_coords_m = grid_coords + (gl2_coords[2]-cell_coord[2])*avnormal
            if (abs(grid_coords_m[index_closest_cell][2]-gl2_coords[2]) >0.2):
                print(grid_coords_m[index_closest_cell], gl2_coords)
        
            ## Move the grid such that is is 1.0 lower than the GL2.
            grid_coords_m = grid_coords_m + depth_max*avnormal ## Upper
            cell_coord = grid_coords_m[index_closest_cell]
        
            ## Get closest index of moved grid:
            index_closest_cell2 = np.argmin((grid_coords_m[:,0] - gl2_coords[0])**2 + (grid_coords_m[:,1] - gl2_coords[1])**2 + (grid_coords_m[:,2] - gl2_coords[2])**2, axis=0 )
            cell_coord2 = grid_coords_m[index_closest_cell2] ## corresponding cell
        
            ## Move positions of beads in molecule such that cell_coords2 = [0, 0, 0], origin
            dr_atom_ref = min_image_dist(res_coords, cell_coord2, box)
        
            ## Flip normal (currently pointed into the membrane) after flipping point out!
            normal_gl2 = normal ##[index_closest_cell]
            # normal_gl2 = -1 * np.mean(normals, axis=0)
        
            ## Calculate the inner product. If the inner product is below zero >> The bead is below the grid 1.0 below gl2
            cos_alpha = (np.inner(dr_atom_ref, normal_gl2) /
                              (np.linalg.norm(dr_atom_ref, axis=1) * 
                               np.linalg.norm(normal_gl2))) # division by norm not necessary, 
                                                            # sign of inner product enough to check depth criterion
            
            for a, atom in enumerate(res.atoms):  #[np.where(cos_alpha >= 0)]:
                atom_name = atom.name ## get atom name to determine whether it is aliphatic or not.
            
                atom_radius, m_value = get_radius_and_defect_type(atom_name, cos_alpha[a])
            
                dist_lim = (dist_clo+atom_radius)**2
                dist_meet = ((dist_far)+atom_radius)**2
            
                ## Move grid to location of lipid:
#             dist_lim = ((np.sqrt(3)/2)+atom_radius)**2
#             dist_meet = (1+atom_radius)**2

                ix_closest = np.argmin((grid_coords_m[:,0] - atom.position[0])**2 + (grid_coords_m[:,1] - atom.position[1])**2 + (grid_coords_m[:,2] - atom.position[2])**2, axis=0 )
                dist = np.sqrt(np.sum((atom.position-grid_coords_m[ix_closest])**2))
                minorplus = np.linalg.norm(atom.position) - np.linalg.norm(grid_coords_m[ix_closest])
                dist = (minorplus / abs(minorplus)) * dist
            
                grid_coords_mm = grid_coords_m + (dist)*avnormal
            
#             if (np.sqrt(np.sum((atom.position-grid_coords_mm[ix_closest])**2))>0.2):
#                 print(atom.position)
#                 print(grid_coords_mm[ix_closest])
#                 print(np.sqrt(np.sum((atom.position-grid_coords_mm[ix_closest])**2)))
            
                delta_x = np.sum((grid_coords_mm-atom.position)**2, axis=1)

                ix_lim = np.where(delta_x <= dist_lim)[0]
                ix_meet = np.where(delta_x <= dist_meet)[0]
            
                ix_meet = np.unique(ix_meet)
                for x1 in ix_lim:
                    if np.isnan(M_1[x1]):
                        if m_value == m_polar:
                            M_1[x1] = 1
                        else:   
                            M_1[x1] = 0
                
                M_1[ix_meet] += m_value
        
        
    return M_1

def fill_in_defect_matrix3(gl2_leaflet, grid_coords, normals, box, depth_max, upper=False, tinygridradius=10, cutoff=0.6):
    m_polar = 1.0
    m_aliphatic = 0.001
    dist_far = cutoff
    dist_clo = cutoff
    ## Initalize matrix with size of number of cells
    M_1 = np.ones(np.shape(grid_coords)[0])
    M_1[:] = np.nan
    upperval = 1
    if (upper == True):
        upperval = -1
        ## v2: added this: (v1, without this part belw.)
        # normals = normals * upperval
    
    ## Loop over all lipids in the leaflet
    for gl2 in gl2_leaflet[:]:
        res = gl2.residue # Get the residue
        gl2_coords = gl2.position # position of the GL2 in A
        res_coords = res.atoms.positions # all positions of all beads in the molecule in A
        
        ## Get index of the closest gridcell to the gl2_coordinate
        index_closest_cell = np.argmin((grid_coords[:,0] - gl2_coords[0])**2 + (grid_coords[:,1] - gl2_coords[1])**2 + (grid_coords[:,2] - gl2_coords[2])**2, axis=0 )
        
        ## Get the corresponding coordinates for the grid cell that is closest to the gl2 coordinate
        cell_coord = grid_coords[index_closest_cell] ## corresponding coordinate
        
        ## get averaged normal vector over small grid of radius 10A 
        normal = np.mean(normals[np.where(np.sum((grid_coords - gl2_coords)**2, axis=1) < tinygridradius**2)], axis=0)
        if np.isnan(normal[0]):
            print(normal)
            pass
        else:
            
            ## Take an average of normal vectors
            avnormal = upperval * normal
            avnormal = avnormal / np.sqrt(np.sum(avnormal**2)) ## Normalize
        
            ## Move grid such that is goes through the GL2 bead of the lipid
            grid_coords_m = grid_coords + (gl2_coords[2]-cell_coord[2])*avnormal
            if (abs(grid_coords_m[index_closest_cell][2]-gl2_coords[2]) >0.2):
                print(grid_coords_m[index_closest_cell], gl2_coords)
                
            ## Move the grid such that is is 1.0 lower than the GL2.
            grid_coords_m = grid_coords_m + depth_max*avnormal ## Upper
            cell_coord = grid_coords_m[index_closest_cell]
        
            ## Get closest index of moved grid:
            index_closest_cell2 = np.argmin((grid_coords_m[:,0] - gl2_coords[0])**2 + (grid_coords_m[:,1] - gl2_coords[1])**2 + (grid_coords_m[:,2] - gl2_coords[2])**2, axis=0 )
            cell_coord2 = grid_coords_m[index_closest_cell2] ## corresponding cell
        
            ## Move positions of beads in molecule such that cell_coords2 = [0, 0, 0], origin
            dr_atom_ref = min_image_dist(res_coords, cell_coord2, box)
        
            ## Flip normal (currently pointed into the membrane) after flipping point out!
            normal_gl2 = normal ##[index_closest_cell]
            # normal_gl2 = -1 * np.mean(normals, axis=0)
        
            ## Calculate the inner product. If the inner product is below zero >> The bead is below the grid 1.0 below gl2
            cos_alpha = (np.inner(dr_atom_ref, normal_gl2) /
                              (np.linalg.norm(dr_atom_ref, axis=1) * 
                               np.linalg.norm(normal_gl2))) # division by norm not necessary, 
                                                            # sign of inner product enough to check depth criterion
            
            for a, atom in enumerate(res.atoms):  #[np.where(cos_alpha >= 0)]:
                atom_name = atom.name ## get atom name to determine whether it is aliphatic or not.
            
                atom_radius, m_value = get_radius_and_defect_type(atom_name, cos_alpha[a])
            
                dist_lim = (dist_clo+atom_radius)**2

                ## Move lipid to grid:
                ix_closest = np.argmin((grid_coords_m[:,0] - atom.position[0])**2 + (grid_coords_m[:,1] - atom.position[1])**2 + (grid_coords_m[:,2] - atom.position[2])**2, axis=0 )
                dist = np.sqrt(np.sum((atom.position-grid_coords_m[ix_closest])**2))
                minorplus = np.linalg.norm(atom.position) - np.linalg.norm(grid_coords_m[ix_closest])
                dist = (minorplus / abs(minorplus)) * dist
                
                pos = atom.position - (dist)*avnormal

                delta_x = np.sum((grid_coords_m-pos)**2, axis=1)

                ix_lim = np.unique(np.where(delta_x <= dist_lim)[0])
                
                for x1 in ix_lim:
                    if np.isnan(M_1[x1]):
                        if m_value == m_polar:
                            M_1[x1] = 1
                        else:   
                            M_1[x1] = 0
                
                M_1[ix_lim] += m_value
        
        
    return M_1

def fill_in_defect_matrix4(gl2_leaflet, grid_coords, normals, box, depth_max, upper=False, tinygridradius=10, cutoff=0.6):
    m_polar = 1.0
    m_aliphatic = 0.001
    dist_far = cutoff
    dist_clo = cutoff
    ## Initalize matrix with size of number of cells
    M_1 = np.ones(np.shape(grid_coords)[0])
    M_1[:] = np.nan
    upperval = 1
    if (upper == True):
        upperval = -1
        ## v2: added this: (v1, without this part belw.)
        # normals = normals * upperval
    
    ## Loop over all lipids in the leaflet
    for gl2 in gl2_leaflet[:]:
        res = gl2.residue # Get the residue
        gl2_coords = gl2.position # position of the GL2 in A
        res_coords = res.atoms.positions # all positions of all beads in the molecule in A
        
        ## Get index of the closest gridcell to the gl2_coordinate
        index_closest_cell = np.argmin((grid_coords[:,0] - gl2_coords[0])**2 + (grid_coords[:,1] - gl2_coords[1])**2 + (grid_coords[:,2] - gl2_coords[2])**2, axis=0 )
        
        ## Get the corresponding coordinates for the grid cell that is closest to the gl2 coordinate
        cell_coord = grid_coords[index_closest_cell] ## corresponding coordinate
        
        ## get averaged normal vector over small grid of radius 10A 
        normal = np.mean(normals[np.where(np.sum((grid_coords - gl2_coords)**2, axis=1) < tinygridradius**2)], axis=0)
        if np.isnan(normal[0]):
            print(normal)
            pass
        else:
            
            ## Take an average of normal vectors
            avnormal = upperval * normal
            avnormal = avnormal / np.sqrt(np.sum(avnormal**2)) ## Normalize
        
            ## Move grid such that is goes through the GL2 bead of the lipid
            grid_coords_m = grid_coords + (gl2_coords-cell_coord)*avnormal
#             if (abs(grid_coords_m[index_closest_cell][2]-gl2_coords[2]) >0.2):
#                 print(grid_coords_m[index_closest_cell], gl2_coords)
                
            ## Move the grid such that is is 1.0 lower than the GL2.
            grid_coords_m = grid_coords_m + depth_max*avnormal ## Upper
            
            ## Get closest index of moved grid:
            index_closest_cell2 = np.argmin((grid_coords_m[:,0] - gl2_coords[0])**2 + (grid_coords_m[:,1] - gl2_coords[1])**2 + (grid_coords_m[:,2] - gl2_coords[2])**2, axis=0 )
            cell_coord2 = grid_coords_m[index_closest_cell2] ## corresponding cell
        
            ## Move positions of beads in molecule such that cell_coords2 = [0, 0, 0], origin
#             dr_atom_ref = min_image_dist(res_coords, cell_coord2, box)
            dr_atom_c = res_coords-cell_coord2
        
            ## Flip normal (currently pointed into the membrane) after flipping point out!
            normal_gl2 = normal ##[index_closest_cell]
        
            ## Calculate the inner product. If the inner product is below zero >> The bead is below the grid 1.0 below gl2
            cos_alpha = np.inner(dr_atom_c, normal_gl2) ## If above 0 >> below surface
            
            for a, atom in enumerate(res.atoms):  #[np.where(cos_alpha >= 0)]:
                atom_name = atom.name ## get atom name to determine whether it is aliphatic or not.
            
                atom_radius, m_value = get_radius_and_defect_type(atom_name, cos_alpha[a])
            
                dist_lim = (dist_clo+atom_radius)**2

                ## Move lipid to grid:
                ix_closest = np.argmin((grid_coords_m[:,0] - atom.position[0])**2 + (grid_coords_m[:,1] - atom.position[1])**2 + (grid_coords_m[:,2] - atom.position[2])**2, axis=0 )
                dist = np.sqrt(np.sum((atom.position-grid_coords_m[ix_closest])**2))
                minorplus = np.linalg.norm(atom.position) - np.linalg.norm(grid_coords_m[ix_closest])
                dist = (minorplus / abs(minorplus)) * dist
                
                pos = atom.position - (dist)*avnormal

                delta_x = np.sum((grid_coords_m-pos)**2, axis=1)

                ix_lim = np.unique(np.where(delta_x <= dist_lim)[0])
                
                for x1 in ix_lim:
                    if np.isnan(M_1[x1]):
                        if m_value == m_polar:
                            M_1[x1] = 1
                        else:   
                            M_1[x1] = 0
                
                M_1[ix_lim] += m_value
        
        
    return M_1

def get_clusters_using_mesh(M_discrete_1, vertices, triangles, minmax):
    xmin, xmax = minmax[0], minmax[1]
    ymin, ymax = minmax[2], minmax[3]
    
    ix_triangles_0 = np.where(M_discrete_1==0)[0]
    ix_triangles_1 = np.where(M_discrete_1==1)[0]
    ix_triangles_01 = np.concatenate((ix_triangles_0, ix_triangles_1))
#     ix_triangles_2 = np.where(M_discrete_1==2)[0]
#     ix_triangles_min1 = np.where(M_discrete_1==-1)[0]
#     ix_triangles_2 = np.concatenate((ix_triangles_2, ix_triangles_min1))
    
    triangles_0 = triangles[ix_triangles_0]
    triangles_1 = triangles[ix_triangles_1]
    triangles_01 = triangles[ix_triangles_01]
#     triangles_2 =  triangles[ix_triangles_2]
    
    # ## Delete triangles at edges:
    # xmin = int(np.min(upper.positions[:,0]))
    # xmax = int(np.max(upper.positions[:,0]))
    # ymin = int(np.min(upper.positions[:,1]))
    # ymax = int(np.max(upper.positions[:,1]))
    
    ix_vertices = np.where(vertices[:,0] <= (xmin+1))[0]
    ix_vertices = np.append(ix_vertices, np.where(vertices[:,0] >= (xmax-1))[0])
    ix_vertices = np.append(ix_vertices, np.where(vertices[:,1] <= (ymin+1))[0])
    ix_vertices = np.append(ix_vertices, np.where(vertices[:,1] >= (ymax-1))[0])
    
    
    ## Delete ix per 0:
    ## get labels
    mesh_0 = o3d.geometry.TriangleMesh()
    mesh_0.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_0.triangles = o3d.utility.Vector3iVector(triangles_0)
    labels, clusters, areas = mesh_0.cluster_connected_triangles()
    labels= np.array(labels)
    
    del_ix = np.array([], dtype=int)
    for i in ix_vertices:
        if i in triangles_0:
            ix=np.where(triangles_0 == i)[0]
#             print(ix)
#             print(labels[ix])
            del_labels = np.unique(labels[ix])
            for dl in del_labels:
                del_ix = np.append(del_ix, np.where(labels == dl)[0])
    del_ix = np.unique(del_ix)
    triangles_0 = np.delete(triangles_0, del_ix, axis=0)
    
    
    ## Delete ix per 1:
    ## get labels
    mesh_1 = o3d.geometry.TriangleMesh()
    mesh_1.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_1.triangles = o3d.utility.Vector3iVector(triangles_1)
    labels, clusters, areas = mesh_1.cluster_connected_triangles()
    labels= np.array(labels)
    
    del_ix = np.array([], dtype=int)
    for i in ix_vertices:
        if i in triangles_1:
            ix=np.where(triangles_1 == i)[0]
            del_labels = np.unique(labels[ix])
            for dl in del_labels:
                del_ix = np.append(del_ix, np.where(labels == dl)[0])
    del_ix = np.unique(del_ix)
    triangles_1 = np.delete(triangles_1, del_ix, axis=0)
    
    ## Delete ix per 01:
    ## get labels
    mesh_01 = o3d.geometry.TriangleMesh()
    mesh_01.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_01.triangles = o3d.utility.Vector3iVector(triangles_01)
    labels, clusters, areas = mesh_01.cluster_connected_triangles()
    labels= np.array(labels)
    
    del_ix = np.array([], dtype=int)
    for i in ix_vertices:
        if i in triangles_01:
            ix=np.where(triangles_01 == i)[0]
            del_labels = np.unique(labels[ix])
            for dl in del_labels:
                del_ix = np.append(del_ix, np.where(labels == dl)[0])
    del_ix = np.unique(del_ix)
    triangles_01 = np.delete(triangles_01, del_ix, axis=0)
    
    ## mesh 0
    mesh_0 = o3d.geometry.TriangleMesh()
    mesh_0.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_0.triangles = o3d.utility.Vector3iVector(triangles_0)
    
    ## mesh 1
    mesh_1 = o3d.geometry.TriangleMesh()
    mesh_1.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_1.triangles = o3d.utility.Vector3iVector(triangles_1)
    
    ## mesh 01
    mesh_01 = o3d.geometry.TriangleMesh()
    mesh_01.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_01.triangles = o3d.utility.Vector3iVector(triangles_01)
    
#     mesh_2 = o3d.geometry.TriangleMesh()
#     mesh_2.vertices = o3d.utility.Vector3dVector(vertices)
#     mesh_2.triangles = o3d.utility.Vector3iVector(triangles_2)
    
    label, clusters, areas0 = mesh_0.cluster_connected_triangles()
    label, clusters, areas1 = mesh_1.cluster_connected_triangles()
    label, clusters, areas01 = mesh_01.cluster_connected_triangles()
#     label, clusters, areas2 = mesh_2.cluster_connected_triangles()
    
    return areas0, areas1, areas01, mesh_0, mesh_1, mesh_01

def get_clusters_using_mesh_stitch(M_discrete_1, vertices, triangles, minmax):
    xmin, xmax = minmax[0], minmax[1]
    ymin, ymax = minmax[2], minmax[3]
    
    ix_triangles_0 = np.where(M_discrete_1==0)[0]
    ix_triangles_1 = np.where(M_discrete_1==1)[0]
    ix_triangles_01 = np.concatenate((ix_triangles_0, ix_triangles_1))

    triangles_0 = triangles[ix_triangles_0]
    triangles_1 = triangles[ix_triangles_1]
    triangles_01 = triangles[ix_triangles_01]
    
    ix_vertices = np.where(vertices[:,0] <= (xmin+1))[0]
    ix_vertices = np.append(ix_vertices, np.where(vertices[:,0] >= (xmax-1))[0])
    ix_vertices = np.append(ix_vertices, np.where(vertices[:,1] <= (ymin+1))[0])
    ix_vertices = np.append(ix_vertices, np.where(vertices[:,1] >= (ymax-1))[0])
    
    
    ## Delete ix per 0:
    ## get labels
    mesh_0 = o3d.geometry.TriangleMesh()
    mesh_0.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_0.triangles = o3d.utility.Vector3iVector(triangles_0)
    labels, clusters, areas = mesh_0.cluster_connected_triangles()
    labels= np.array(labels)
    
    del_ix = np.array([], dtype=int)
    for i in ix_vertices:
        if i in triangles_0:
            ix=np.where(triangles_0 == i)[0]
#             print(ix)
#             print(labels[ix])
            del_labels = np.unique(labels[ix])
            for dl in del_labels:
                del_ix = np.append(del_ix, np.where(labels == dl)[0])
    del_ix = np.unique(del_ix)
    triangles_0 = np.delete(triangles_0, del_ix, axis=0)
    
    ## Delete ix per 1:
    ## get labels
    mesh_1 = o3d.geometry.TriangleMesh()
    mesh_1.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_1.triangles = o3d.utility.Vector3iVector(triangles_1)
    labels, clusters, areas = mesh_1.cluster_connected_triangles()
    labels= np.array(labels)
    
    del_ix = np.array([], dtype=int)
    for i in ix_vertices:
        if i in triangles_1:
            ix=np.where(triangles_1 == i)[0]
            del_labels = np.unique(labels[ix])
            for dl in del_labels:
                del_ix = np.append(del_ix, np.where(labels == dl)[0])
    del_ix = np.unique(del_ix)
    triangles_1 = np.delete(triangles_1, del_ix, axis=0)
    
    ## Delete ix per 01:
    ## get labels
    mesh_01 = o3d.geometry.TriangleMesh()
    mesh_01.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_01.triangles = o3d.utility.Vector3iVector(triangles_01)
    labels, clusters, areas = mesh_01.cluster_connected_triangles()
    labels= np.array(labels)
    
    del_ix = np.array([], dtype=int)
    for i in ix_vertices:
        if i in triangles_01:
            ix=np.where(triangles_01 == i)[0]
            del_labels = np.unique(labels[ix])
            for dl in del_labels:
                del_ix = np.append(del_ix, np.where(labels == dl)[0])
    del_ix = np.unique(del_ix)
    triangles_01 = np.delete(triangles_01, del_ix, axis=0)
    
    ## mesh 0
    mesh_0 = o3d.geometry.TriangleMesh()
    mesh_0.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_0.triangles = o3d.utility.Vector3iVector(triangles_0)
    
    ## mesh 1
    mesh_1 = o3d.geometry.TriangleMesh()
    mesh_1.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_1.triangles = o3d.utility.Vector3iVector(triangles_1)
    
    ## mesh 01
    mesh_01 = o3d.geometry.TriangleMesh()
    mesh_01.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_01.triangles = o3d.utility.Vector3iVector(triangles_01)
    
    areas0 = stitch_clusters(mesh_0)
    areas1 = stitch_clusters(mesh_1)
    areas01 = stitch_clusters(mesh_01)
    
    return areas0, areas1, areas01, mesh_0, mesh_1, mesh_01


def make_meshgrid_for_imaging_from_mesh(cot0, cot1, vertices, edge, pixel):
# M_discrete = M_discrete_1_up
# grid_coords = grid_coords_up
# edge = 5
# pixel = 1
    
    minx, maxx = np.min(vertices[:,0]), np.max(vertices[:,0])
    miny, maxy = np.min(vertices[:,1]), np.max(vertices[:,1])
    xx = np.arange(np.floor(minx-edge), np.ceil(maxx+edge), pixel)
    yy = np.arange(np.floor(miny-edge), np.ceil(maxy+edge), pixel)
 
    meshgrid = np.meshgrid(xx, yy)[0]
    meshgrid[:] = 1
    
    for coor in cot0:
        xix = np.where((xx - coor[0])**2 == np.min((xx - coor[0])**2))[0][0]
        yix = np.where((yy - coor[1])**2 == np.min((yy - coor[1])**2))[0][0]
            
        meshgrid[yix][xix] = 0 
    
    for coor in cot1:
        xix = np.where((xx - coor[0])**2 == np.min((xx - coor[0])**2))[0][0]
        yix = np.where((yy - coor[1])**2 == np.min((yy - coor[1])**2))[0][0]
            
        meshgrid[yix][xix] = 2
        
    
    return meshgrid, xx, yy

def make_meshgrid_for_imaging_delete_edge(label_0, label_1, grid_coords, edge, pixel):
# M_discrete = M_discrete_1_up
# grid_coords = grid_coords_up
# edge = 5
# pixel = 1

    minx, maxx = np.min(grid_coords[:,0]), np.max(grid_coords[:,0])
    miny, maxy = np.min(grid_coords[:,1]), np.max(grid_coords[:,1])
    xx = np.arange(np.floor(minx-edge), np.ceil(maxx+edge), pixel)
    yy = np.arange(np.floor(miny-edge), np.ceil(maxy+edge), pixel)
 
    meshgrid = np.meshgrid(xx, yy)[0]
    meshgrid[:] = 1
    
    for m, mval in enumerate(label_0[:]):
        if mval != 0:
            coor = grid_coords[m]
            xix = np.where((xx - coor[0])**2 == np.min((xx - coor[0])**2))[0][0]
            yix = np.where((yy - coor[1])**2 == np.min((yy - coor[1])**2))[0][0]
            
            meshgrid[yix][xix] = 0 
            
    for m, mval in enumerate(label_1[:]):
        if mval != 0:
            coor = grid_coords[m]
            xix = np.where((xx - coor[0])**2 == np.min((xx - coor[0])**2))[0][0]
            yix = np.where((yy - coor[1])**2 == np.min((yy - coor[1])**2))[0][0]
            
            meshgrid[yix][xix] = 2
    return meshgrid, xx, yy

def make_meshgrid_for_imaging(M_discrete, grid_coords, edge, pixel):
# M_discrete = M_discrete_1_up
# grid_coords = grid_coords_up
# edge = 5
# pixel = 1

    minx, maxx = np.min(grid_coords[:,0]), np.max(grid_coords[:,0])
    miny, maxy = np.min(grid_coords[:,1]), np.max(grid_coords[:,1])
    xx = np.arange(np.floor(minx-edge), np.ceil(maxx+edge), pixel)
    yy = np.arange(np.floor(miny-edge), np.ceil(maxy+edge), pixel)
 
    meshgrid = np.meshgrid(xx, yy)[0]
    meshgrid[:] = 1
    
    for m, mval in enumerate(M_discrete[:]):
        if mval == 0:
            coor = grid_coords[m]
            xix = np.where((xx - coor[0])**2 == np.min((xx - coor[0])**2))[0][0]
            yix = np.where((yy - coor[1])**2 == np.min((yy - coor[1])**2))[0][0]
            
            meshgrid[yix][xix] = 0 
            
        if mval == 1:
            coor = grid_coords[m]
            xix = np.where((xx - coor[0])**2 == np.min((xx - coor[0])**2))[0][0]
            yix = np.where((yy - coor[1])**2 == np.min((yy - coor[1])**2))[0][0]
            
            meshgrid[yix][xix] = 2
    return meshgrid, xx, yy


def make_meshgrid_for_imaging_delete_edge(label_0, label_1, grid_coords, edge, pixel):
# M_discrete = M_discrete_1_up
# grid_coords = grid_coords_up
# edge = 5
# pixel = 1

    minx, maxx = np.min(grid_coords[:,0]), np.max(grid_coords[:,0])
    miny, maxy = np.min(grid_coords[:,1]), np.max(grid_coords[:,1])
    xx = np.arange(np.floor(minx-edge), np.ceil(maxx+edge), pixel)
    yy = np.arange(np.floor(miny-edge), np.ceil(maxy+edge), pixel)
 
    meshgrid = np.meshgrid(xx, yy)[0]
    meshgrid[:] = 1
    
    for m, mval in enumerate(label_0[:]):
        if mval != 0:
            coor = grid_coords[m]
            xix = np.where((xx - coor[0])**2 == np.min((xx - coor[0])**2))[0][0]
            yix = np.where((yy - coor[1])**2 == np.min((yy - coor[1])**2))[0][0]
            
            meshgrid[yix][xix] = 0 
            
    for m, mval in enumerate(label_1[:]):
        if mval != 0:
            coor = grid_coords[m]
            xix = np.where((xx - coor[0])**2 == np.min((xx - coor[0])**2))[0][0]
            yix = np.where((yy - coor[1])**2 == np.min((yy - coor[1])**2))[0][0]
            
            meshgrid[yix][xix] = 2
    return meshgrid, xx, yy


## Functions in functions
def min_image_dist(a, b, box):
    d = a - b
    return d - box * np.rint(d / box)


def get_radius_and_defect_type(atom_name, cos_alpha):
    m_polar = 1.0
    m_aliphatic = 0.001
## Check radius and type
    if atom_name in ['NC3', 'NH3']: 
        atom_radius = 3.4796
    elif atom_name in ['PO4']:
        atom_radius = 3.4796
    elif atom_name in ['GL1']:
        atom_radius = 2.413
    else:
        atom_radius = 2.638
    
    if atom_name in ['NC3', 'NH3', 'PO4', 'GL1', 'GL2', 'GLA', 'NAB']:
        m_value = m_polar
    elif cos_alpha > 0.0:
#     elif cos_alpha < 0.0: ## deep defect based on z
        m_value = 0.0
    else:
        m_value = m_aliphatic
        
    return atom_radius, m_value


## Functions 
def make_grid_flat_test(positions, lower=True, radius_norms=100, max_nn=30): 

    ## Make triangle mesh from pointcloud:
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(positions[:,:3])

    ## Compute normals of the points:
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_norms, max_nn=max_nn))
    # pcd.orient_normals_consistent_tangent_plane(100) ## THIS GIVES CORRECT NORMAL VECTORS!  ## DOES NOT WORK FOR THIS DONT KNOW WHY
    pcd.normalize_normals()

    normals = np.asarray(pcd.normals)
    ix_nega = np.where(normals[:,2] < 0)
    normals[ix_nega] = normals[ix_nega]* -1
    pcd.normals = o3d.utility.Vector3dVector(normals)

    if lower==True:
        normals = -1*np.asarray(pcd.normals)
        pcd.normals = o3d.utility.Vector3dVector(normals)
    
    ## Make surface with poisson: 
    mesh_pois, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)
    
    # Check if mesh is continuous, no manifold edges, watertight: 
    mesh_pois.remove_non_manifold_edges()
    mesh_pois.remove_duplicated_vertices()
    mesh_pois.remove_duplicated_triangles()
    
    mesh_pois.compute_triangle_normals()
    mesh_pois.normalize_normals()
    return mesh_pois, pcd

def make_grid_flat_test2(positions, lower=True, radius_norms=100, max_nn=30): 

    ## Make triangle mesh from pointcloud:
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(positions[:,:3])

    ## Compute normals of the points:
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_norms, max_nn=max_nn))
    # pcd.orient_normals_consistent_tangent_plane(100) ## THIS GIVES CORRECT NORMAL VECTORS!  ## DOES NOT WORK FOR THIS DONT KNOW WHY
    pcd.normalize_normals()

    normals = np.asarray(pcd.normals)
    ix_nega = np.where(normals[:,2] < 0)
    normals[ix_nega] = normals[ix_nega]* -1
    pcd.normals = o3d.utility.Vector3dVector(normals)

    if lower==True:
        normals = -1*np.asarray(pcd.normals)
        pcd.normals = o3d.utility.Vector3dVector(normals)
    
    ## Make surface with poisson: 
    mesh_pois, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)
    
    with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
        triangle_clusters, cluster_n_triangles, cluster_area = (
            mesh_pois.cluster_connected_triangles())
    triangle_clusters = np.asarray(triangle_clusters)
    cluster_n_triangles = np.asarray(cluster_n_triangles)
    cluster_area = np.asarray(cluster_area)
    
    triangles_to_remove = cluster_n_triangles[triangle_clusters] < 100
    mesh_pois.remove_triangles_by_mask(triangles_to_remove)
    
    # Check if mesh is continuous, no manifold edges, watertight: 
    mesh_pois.remove_non_manifold_edges()
    mesh_pois.remove_duplicated_vertices()
    mesh_pois.remove_duplicated_triangles()
    
    mesh_pois.compute_triangle_normals()
    mesh_pois.normalize_normals()
    return mesh_pois, pcd



def make_grid_continues_test(positions, radius_norms=100, max_nn=30): 

    ## Make triangle mesh from pointcloud:
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(positions[:,:3])

    ## Compute normals of the points:
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_norms, max_nn=max_nn))

    # pcd.orient_normals_consistent_tangent_plane(100) ## THIS GIVES CORRECT NORMAL VECTORS!  ## DOES NOT WORK FOR THIS DONT KNOW WHY
    pcd.normalize_normals()

    normals = np.asarray(pcd.normals)
    ix_nega = np.where(normals[:,2] < 0)
    normals[ix_nega] = normals[ix_nega]* -1
    pcd.normals = o3d.utility.Vector3dVector(normals)

    pcd.orient_normals_consistent_tangent_plane(111)
    ## Make surface with poisson: 
    mesh_pois, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=7)
    
    # Check if mesh is continuous, no manifold edges, watertight: 
    mesh_pois.remove_non_manifold_edges()
    mesh_pois.remove_duplicated_vertices()
    mesh_pois.remove_duplicated_triangles()
    
    mesh_pois.compute_triangle_normals()
    mesh_pois.normalize_normals()
    return mesh_pois, pcd


def grid_plane_triangulated(grid_l1, size_triangle=1.0, num_it1=1, num_it2=1):
    ## delete: this is outside of func
    current_n_triangles = np.asarray(grid_l1.triangles).shape[0]

    n_triangles = int(grid_l1.get_surface_area()/size_triangle)

    versions_of_4 = int(np.ceil(( n_triangles/current_n_triangles )**(1/4)))
    n_triangles_first = int(n_triangles/ (4**versions_of_4))

    mesh_smp = grid_l1.simplify_quadric_decimation(
        target_number_of_triangles=n_triangles_first)
    
    mesh_smp.remove_non_manifold_edges()
    vertices=mesh_smp.vertices
    triangles=mesh_smp.triangles
    points, cells = optimesh.optimize_points_cells(vertices, triangles, "cvt full", 1.0e-9, num_it1)

    mesh_new = o3d.geometry.TriangleMesh()
    mesh_new.vertices=o3d.utility.Vector3dVector(points[:, :3])
    mesh_new.triangles=o3d.utility.Vector3iVector(cells[:, :3])

    mesh_new.compute_triangle_normals()
    mesh_new.normalize_normals()

    grid_l1_new = o3d.geometry.TriangleMesh.subdivide_midpoint(mesh_new, versions_of_4)
    grid_l1_new.paint_uniform_color([0.9, 0.9, 0.9])

    vertices=grid_l1_new.vertices
    triangles=grid_l1_new.triangles
    # mesh_new = meshplex.MeshTri(vertices, triangles)
    # optimesh.optimize(mesh_new, "cvt full", 1.0e-5, 3)
    # points = mesh_new.points
    # cells = mesh_new.cells
    points, cells = optimesh.optimize_points_cells(vertices, triangles, "cvt full", 1.0e-5, num_it2)

    mesh_new2 = o3d.geometry.TriangleMesh()
    mesh_new2.vertices=o3d.utility.Vector3dVector(points[:, :3])
    mesh_new2.triangles=o3d.utility.Vector3iVector(cells[:, :3])
    
    mesh_new2.compute_triangle_normals()
    mesh_new2.normalize_normals()

    return mesh_new2

def get_cluster_areas(regrid, defectmatrix, minmax):
    xmin, xmax = minmax[0], minmax[1]
    ymin, ymax = minmax[2], minmax[3]
    triangles = np.asarray(regrid.triangles)
    vertices = np.asarray(regrid.vertices)
    
    triangles_1 = np.asarray(triangles)[np.where(defectmatrix==1)]
    triangles_0 = np.asarray(triangles)[np.where(defectmatrix==0)]
    triangles_01 = np.vstack((triangles_1, triangles_0))
    verts = np.asarray(vertices)
    
    ## Mesh 0        
    ## Delete clusters that are on the edge
    ix_vertices = np.array([], dtype=int)
    ix_vertices = np.append(ix_vertices, np.where(vertices[:,0] <= xmin)[0])
    ix_vertices = np.append(ix_vertices, np.where(vertices[:,0] >= xmax)[0])
    ix_vertices = np.append(ix_vertices, np.where(vertices[:,1] <= ymin)[0])
    ix_vertices = np.append(ix_vertices, np.where(vertices[:,1] >= ymax)[0])
    ix_vertices = np.unique(ix_vertices)

    triangles_0_to_del = np.arange(0, len(triangles_0), 1, dtype=int)
    for i in ix_vertices:
        ixtemp = np.where(triangles_0 == i)[0]
        for j in ixtemp:
            triangles_0_to_del = np.delete(triangles_0_to_del, np.where(triangles_0_to_del==j))

    triangles_0_deleted_edges = triangles_0[triangles_0_to_del]
    
    mesh_0_big_del = o3d.geometry.TriangleMesh()
    mesh_0_big_del.vertices = o3d.utility.Vector3dVector(verts[:,:3])
    mesh_0_big_del.triangles = o3d.utility.Vector3iVector(triangles_0_deleted_edges[:,:3])
    
    ## Mesh 1
    ## Delete clusters that are on the edge
    triangles_1_to_del = np.arange(0, len(triangles_1), 1, dtype=int)
    for i in ix_vertices:
        ixtemp = np.where(triangles_1 == i)[0]
        for j in ixtemp:
            triangles_1_to_del = np.delete(triangles_1_to_del, np.where(triangles_1_to_del==j))

    triangles_1_deleted_edges = triangles_1[triangles_1_to_del]
    mesh_1_big_del = o3d.geometry.TriangleMesh()
    mesh_1_big_del.vertices = o3d.utility.Vector3dVector(verts[:,:3])
    mesh_1_big_del.triangles = o3d.utility.Vector3iVector(triangles_1_deleted_edges[:,:3])

    ## Mesh 01
    ## Delete clusters that are on the edge
    triangles_01_to_del = np.arange(0, len(triangles_01), 1, dtype=int)
    for i in ix_vertices:
        ixtemp = np.where(triangles_01 == i)[0]
        for j in ixtemp:
            triangles_01_to_del = np.delete(triangles_01_to_del, np.where(triangles_01_to_del==j))

    triangles_01_deleted_edges = triangles_01[triangles_01_to_del]
    mesh_01_big_del = o3d.geometry.TriangleMesh()
    mesh_01_big_del.vertices = o3d.utility.Vector3dVector(verts[:,:3])
    mesh_01_big_del.triangles = o3d.utility.Vector3iVector(triangles_01_deleted_edges[:,:3])
    
    labs, sizes0, areas0 = mesh_0_big_del.cluster_connected_triangles()
    labs, sizes1, areas1 = mesh_1_big_del.cluster_connected_triangles()
    labs, sizes01, areas01 = mesh_01_big_del.cluster_connected_triangles()
    
    return areas0, areas1, areas01, sizes0, sizes1, sizes01

def get_clustering_0_or_1(M_discrete_1, mval, gridcoords_nbrs, startval=0):
    nbrs = NearestNeighbors(n_neighbors=9).fit(gridcoords_nbrs)
    distances, indices = nbrs.kneighbors(gridcoords_nbrs)
    
    connect_label = {}
    label_0 = np.zeros_like(M_discrete_1[startval:])
    label_dict_0 = {} 
    label_num_new=0
    for mm, m in enumerate(M_discrete_1[startval:]):
        if m==mval:
            if label_0[mm] == 0:
#                 print(mm, m)
                defect_numbers = M_discrete_1[indices[mm]]
                ix = np.where(defect_numbers == mval)
                ixx = indices[mm][ix]
                ix_got_labels = np.unique(np.where(np.unique(label_0[ixx]) != 0))
#                 print(ix_got_labels)
#                 print(ix_got_labels)
                if len(ix_got_labels) == 0: 
                    label_num_new += 1
                    label_num = label_num_new 
                    label_dict_0[label_num] = np.array(ixx)
                    label_0[ixx] = label_num
                elif len(ix_got_labels) == 1:
                    label_num = np.unique(label_0[ixx])[ix_got_labels][0]
                    label_dict_0[label_num] = np.unique(np.append(label_dict_0[label_num], np.array(ixx)))
                    label_0[ixx] = label_num
#                     print(label_num)
                else:
                    label_num = np.min(np.unique(label_0[ixx])[ix_got_labels])
    #                 print(label_num)
                    if  label_num not in connect_label.keys():
                        connect_label[label_num] = np.array(np.unique(label_0[ixx])[ix_got_labels])
                        label_dict_0[label_num] = np.array(ixx)
                    else:
                        connect_label[label_num] = np.append(connect_label[label_num], np.array(np.unique(label_0[ixx])[ix_got_labels]))
                        label_dict_0[label_num] = np.append(label_dict_0[label_num], np.array(ixx))
                
                    label_0[ixx] = label_num
    return label_0, label_dict_0, connect_label


def connect_clusters(connect_labels, labels):
    for k in connect_labels.keys():
        arr = np.unique(connect_labels[k])
        arr = np.sort(arr)
        for a in arr[1:]:
            labels[np.where(labels == a)] = arr[0]
            
    areas_per_cluster = np.array([])
    clusterlabels = np.unique(labels[np.where(labels != 0)])
    
    return labels

def delete_cluster_on_edge(labels, gridcoords, edges):
    xmin, xmax = edges[0], edges[1]
    ymin, ymax = edges[2], edges[3]
    label_numbers = np.unique(labels)
    for ln in label_numbers:
        ix_coors = np.where(labels == ln)
        coordinates = gridcoords[ix_coors]
        minboolx = coordinates[:,0] < xmin
        maxboolx = coordinates[:,0] > xmax
        minbooly = coordinates[:,1] < ymin
        maxbooly = coordinates[:,1] > xmax
        boolarr = np.concatenate((minboolx, maxboolx, minbooly, maxbooly))
        
        if True in boolarr:
            labels[np.where(labels==ln)]= 0
    return labels


def write_file(arr, infile, replacement_line):
    replace_first_line(infile, infile, replacement_line)
    inf = open(infile, 'a')
    for a in arr:
        inf.write('%f\n'%a)
    inf.close()
    return 0

def replace_first_line(src_filename, target_filename, replacement_line):
    f = open(src_filename)
    first_line, remainder = f.readline(), f.read()
    f.close()
    spline = first_line.split(' ')
    count = int(spline[-1])
#     print(count)
    count+=1
    replacement_line += ' %d'%count
    t = open(target_filename, "w")
    t.write(replacement_line+'\n')
    t.write(remainder)
    t.close()
    return 0

def area_fit(area, b, pack_def_const):
    return b * np.exp( -area / pack_def_const )

def log_area_fit(area, b, pack_def_const):
    return np.log(b) - area / pack_def_const



def get_centroid_of_triangles(vertices, triangles): 
    cot = np.zeros_like(triangles, dtype=np.double)
    for t, tr in enumerate(triangles):
        a = np.linalg.norm(vertices[tr[2]]-vertices[tr[1]])
        b = np.linalg.norm(vertices[tr[2]]-vertices[tr[0]])
        c = np.linalg.norm(vertices[tr[1]]-vertices[tr[0]])
        cot[t] = ((a*vertices[tr[0]])+(b*vertices[tr[1]])+(c*vertices[tr[2]]))/(a+b+c)
    return cot 

def get_selection_phaseseparated(gro, cutoff_l, cutoff2):
    selection = 'name GL2 or name GL1'
#     cutoff_l = 15
    L = LeafletFinder(gro, selection, cutoff=cutoff_l, pbc=True)
    sizes = np.array([l.n_atoms for l in L.groups()])
    ix1 = np.where(sizes==np.max(sizes))[0][0]
    sizes[ix1] = 0
    ix2 = np.where(sizes == np.max(sizes))[0][0]
    sizes[ix2] = 0
    
    upper = L.group(ix1)
    lower = L.group(ix2)
    
    if np.mean(upper.positions[:,2]) < np.mean(lower.positions[:,2]):
        upper = L.group(ix2)
        lower = L.group(ix1)
    
    ## Add Phase separated lipid
    selectiongla = 'name ROH'
    gla = gro.select_atoms(selectiongla)
    distup = distance_array(gla.positions, upper.positions)
    distlo = distance_array(gla.positions, lower.positions)
    
    mindistup = np.min(distup, axis=1)
    mindistlo = np.min(distlo, axis=1)

    glares = gla.resids
    upres = upper.resids
    lores = lower.resids
    
#     cutoff2 = 12
    ix_up = np.where(mindistup < cutoff2)
    ix_lo = np.where(mindistlo < cutoff2)
    
    ## Get selection:
    selection_up = '(name ROH or name GL2) and (resid '
    for u in glares[ix_up]:
        selection_up += '%d '%u
    for u in upres:
        selection_up += '%d '%u
    selection_up += ')'

    selection_lo = '(name ROH or name GL2) and (resid '
    for u in glares[ix_lo]:
        selection_lo += '%d '%u
    for u in lores:
        selection_lo += '%d '%u
    selection_lo += ')'
    
    upper = gro.select_atoms(selection_up)
    lower = gro.select_atoms(selection_lo)
    return upper, lower

def add_pbc_egdes(pos, box, edge_val = 7):
        
    ix_add_x_border = np.where(pos[:,0] < (np.min(pos[:,0]+edge_val)))[0]
    ix_min_x_border = np.where(pos[:,0] > (np.max(pos[:,0]-edge_val)))[0]
    
    ix_add_y_border = np.where(pos[:,1] < (np.min(pos[:,1]+edge_val)))[0]
    ix_min_y_border = np.where(pos[:,1] > (np.max(pos[:,1]-edge_val)))[0]

    addx = pos[ix_add_x_border] + [box[0], 0, 0]
    minx = pos[ix_min_x_border] - [box[0], 0, 0]
    
    addy = pos[ix_add_y_border] + [0, box[1], 0]
    miny = pos[ix_min_y_border] - [0, box[1], 0]
    
    newpos = np.concatenate((pos, addx, minx, addy, miny))
    
    return newpos

## area distribution:
def herons_formula(a, b, c):
    ab = np.linalg.norm(b-a)
    bc = np.linalg.norm(b-c)
    ac = np.linalg.norm(c-a)
    
    s = (ab+bc+ac)/2
    return np.sqrt(s*(s-ab)*(s-bc)*(s-ac))

def areas_triangles_distribution(vertices, triangles):
    areas = np.zeros(len(triangles[:,0]))
    for t, tr in enumerate(triangles):
        areas[t] = herons_formula(vertices[tr[0]], vertices[tr[1]], vertices[tr[2]])
    bins = np.arange(0, 10, 0.05)
    hist = np.histogram(areas, bins, density=True)
    binsreal = (bins[:-1]+bins[1:])/2
    return hist[0], binsreal

def average_distance_neighbours(gridcoords_nbrs, n_neighbors):
    nbrs = NearestNeighbors(n_neighbors=(n_neighbors+1)).fit(gridcoords_nbrs)
    distances, indices = nbrs.kneighbors(gridcoords_nbrs)
    return distances

def plot_histogram_2_surfaces(vertices_up, triangle_up, vertices_lo, triangle_lo, savedir, savename):
    histup, binsup = areas_triangles_distribution(vertices_up, triangle_up)
    histlo, binslo = areas_triangles_distribution(vertices_lo, triangle_lo)
    
    ## plot:
    fig = plt.figure(figsize=(3, 3))
    ax = plt.subplot(1, 1, 1)
    use_in_fit = np.where(histup > 0.01)
    ax.plot(binsup[use_in_fit], histup[use_in_fit], c=colors[1], lw=2.5, label='Upper leaflet')
    
    use_in_fit = np.where(histlo > 0.01)
    ax.plot(binslo[use_in_fit], histlo[use_in_fit], c=colors[3], lw=2.5, label='Lower leaflet')

    ax.legend(frameon=False)
    ax.set_xlabel('Probability', fontsize=11)
    ax.set_ylabel('Area of triangles', fontsize=11)
    
    plt.savefig(savedir+savename+'.svg', bbox_inches='tight')
    
    ## plottingfile:
    inf = open(savedir+savename+'.txt', 'w')
    inf.write('bins up, histup, bins lo, hist lo\n')
    for i in range(0, len(histup)):
        inf.write('%.3f  %.3f  %.3f  %.3f\n'%(binsup[i], histup[i], binslo[i], histlo[i]))
    inf.close()
    return 0

def plot_histogram_1_surface(vertices, triangle, savedir, savename):
    histup, binsup = areas_triangles_distribution(vertices, triangle)
    
    ## plot:
    fig = plt.figure(figsize=(3, 3))
    ax = plt.subplot(1, 1, 1)
    use_in_fit = np.where(histup > 0.01)
    ax.plot(binsup[use_in_fit], histup[use_in_fit], c=colors[1], lw=2.5) 
    
    ax.legend(frameon=False)
    ax.set_xlabel('Probability', fontsize=11)
    ax.set_ylabel('Area of triangles', fontsize=11)
    
    plt.savefig(savedir+savename+'.svg', bbox_inches='tight')
    
    ## plottingfile:
    inf = open(savedir+savename+'.txt', 'w')
    inf.write('bins histogram\n')
    for i in range(0, len(histup)):
        inf.write('%.3f  %.3f \n'%(binsup[i], histup[i]))
    inf.close()
    return 0
    
def stitch_clusters(mesh_1):
	mesh_1.remove_unreferenced_vertices()
	labels, clusters, areas = mesh_1.cluster_connected_triangles()
	triangles_1 = np.asarray(mesh_1.triangles)
	newareas = np.array([])
	labels = np.asarray(labels)
	areas = np.asarray(areas)
	skipping = np.array([])
	for l, lab in enumerate(labels):
		if lab in skipping: 
			pass
		else:
		    	ix = np.where(labels == lab)
    			vert_lab = triangles_1[ix].flatten() # vertices related to label
    			ix = np.array([],dtype=int) # get ix for all triangles that share vertex
    			for v in vert_lab:
    				ix = np.append(ix, np.where(triangles_1 == v)[0]) ## get index for all triangles with this vertex
    			ix = np.unique(ix) ## ix for triangles and labels
	    		labeltemp = labels[ix]
	    		if len(np.unique(labeltemp)) != 1:
    				areastemp = 0
    				for cl in np.unique(labeltemp):
    					skipping = np.append(skipping, cl)
    					areastemp += areas[cl]
    					## merch labels: 
    				newareas = np.append(newareas, areastemp)
    			else:
    				newareas = np.append(newareas, areas[labeltemp[0]])
    				skipping = np.append(skipping, labeltemp[0])
	return newareas

def get_cutoff(distup, distlo, n_neighbors):
    ix_up = np.where(distup[:,n_neighbors] < (np.mean(distup[:,n_neighbors])+np.std(distup[:,n_neighbors])))
    ix_lo = np.where(distlo[:,n_neighbors] < (np.mean(distlo[:,n_neighbors])+np.std(distlo[:,n_neighbors])))
    
    dist = np.array([])
    
    for i in range(1, (n_neighbors+1)):
        dist = np.append(dist, distup[:,i][ix_up])
        dist = np.append(dist, distlo[:,i][ix_lo])
        
        if i == 3:
            cutoff3 = np.mean(dist)
    cutoff_n = np.mean(dist)
    
    return cutoff3, cutoff_n

def initialize_variables():
    keys = ['area_of_all_defects', 'number_of_defect_all_triangles', 'total_number_of_triangles', 'number_of_clusters', 'total_surface_area_mesh', 'total_surface_area_box', 'mean_area_clusters', 'perc_triangles_on_edge', 'total_area_all_defects']
    variables_up = {}
    variables_lo = {}

    for k in keys:
        variables_up[k] = np.array([])
        variables_lo[k] = np.array([])
    return variables_up, variables_lo, keys

def put_in_variables(variables_up, keys, areas01, mesh_01, grid_coords, regrid, box, pos):
    variables_up[keys[0]] = np.append(variables_up[keys[0]], np.sum(areas01))
    variables_up[keys[1]] = np.append(variables_up[keys[1]], len(np.asarray(mesh_01.triangles)))
    variables_up[keys[2]] = np.append(variables_up[keys[2]], len(grid_coords))
    variables_up[keys[3]] = np.append(variables_up[keys[3]], len(areas01))
    variables_up[keys[4]] = np.append(variables_up[keys[4]], regrid.get_surface_area())
    variables_up[keys[5]] = np.append(variables_up[keys[5]], box[0]*box[1])
    variables_up[keys[6]] = np.append(variables_up[keys[6]], np.mean(areas01))
    
    ix_triangle_out = np.where(np.asarray(regrid.vertices)[:, 0] < np.min(pos[:, 0])- 3)
    ix_triangle_out = np.append(ix_triangle_out, np.where(np.asarray(regrid.vertices)[:, 0] > np.max(pos[:, 0])+ 3))
    ix_triangle_out = np.append(ix_triangle_out, np.where(np.asarray(regrid.vertices)[:, 1] < np.min(pos[:, 1])- 3))
    ix_triangle_out = np.append(ix_triangle_out, np.where(np.asarray(regrid.vertices)[:, 1] > np.max(pos[:, 1])+ 3))
    
    variables_up[keys[7]] = np.append(variables_up[keys[7]], len(np.asarray(regrid.triangles)[np.unique(ix_triangle_out)])/ len(np.asarray(regrid.triangles)))
    variables_up[keys[8]] = np.append(variables_up[keys[8]], len(areas01))
    return variables_up

In [4]:
## Get right directories
basedir = '/Users/ankush/Desktop/Ankush/C60-Chol/Defects/flat-325K-membrane-only/more-radius/'
# workdir = basedir + 'Pproject/analysis/packmem_new/validation_box_sizes/DPPC/box_30/'
# savedir = basedir + 'Pproject/analysis/packmem_new/validation_box_sizes/DPPC/box_30/png/'
# infiletpr = 'step9.tpr'
# infilextc = 'step9.xtc'

workdir = '/Users/ankush/Desktop/Ankush/C60-Chol/Defects/flat-325K-membrane-only/more-radius/'
savedir = '/Users/ankush/Desktop/Ankush/C60-Chol/Defects/flat-325K-membrane-only/more-radius/'
# # workdir = basedir + 'Pproject/analysis/packmem_new/validation_box_sizes/buckled/buckled_DPPC/gerelaxet/'
# # savedir = basedir + 'Pproject/analysis/packmem_new/validation_box_sizes/buckled/buckled_DPPC/gerelaxet/png/'

# workdir = basedir + 'Winant/martini2/buckles/2_DSPG_CHOL/T325/'
# savedir = basedir + 'Winant/martini2/buckles/2_DSPG_CHOL/T325/TriMesh/'
# infiletpr = 'buckle_dspgchol_prod325.tpr'
# # infilextc = 'buckle_dspgchol_prod325.xtc'
# infilextc = 'buckle_connect_dspgchol325.xtc'

# infiletpr = 'DPPC-buckled-285K.tpr'
# infilextc = 'DPPC_buckled_285K_shorter.xtc'
infiletpr = 'DPPC.tpr'
infilextc = 'DPPC.xtc'

depth_max=4.5
prefix = 'areas'
affix = '_dppc303'

print('Read trajectory')
gro = mda.Universe(workdir + infiletpr, workdir + infilextc)

## Sytem variables
lx = gro.dimensions[0] 
ly = gro.dimensions[1] 
lz = gro.dimensions[2] 
box = gro.dimensions[:3] 

Read trajectory


In [5]:
workdir, savedir #, prefix, affixup

('/Users/ankush/Desktop/Ankush/C60-Chol/Defects/flat-325K-membrane-only/more-radius/',
 '/Users/ankush/Desktop/Ankush/C60-Chol/Defects/flat-325K-membrane-only/more-radius/')

In [6]:
## ## VARIABLES

## Grid fitting:
striangle=1.0
cutoff = 1.3
numit1 = 6
numit2 = 5

## Imaging
imaging = True
everyframe=50
edge=5
pixel=1.2

## Trajectory
skip_arr = np.array([])

## Defect matrix
depth_max_lo = depth_max
depth_max_up = -1.0 * depth_max_lo

## Plotting colors
colors = ['#001219', '#005F73', '#0A9396', '#94D2BD', '#E9D8A6', '#EE9B00', '#CA6702', '#BB3E03', '#AE2012', '#9B2226']


print(len(gro.trajectory))

# inf = open(savedir+'settings%s%.1f_c%.1f.txt'%(affix,striangle,cutoff), 'w')

# inf.write('size triangle: %f \n'%striangle)
# inf.write('number of iterations 1: %d  & 2: %d \n'%(numit1, numit2))
# inf.write('depth_max: %f \n'%depth_max)
# inf.write('savedir: %s \n'%savedir)
# inf.write('inputpath: %s \n'%workdir)
# inf.write('inputfile: %s & %s \n'%(infiletpr, infilextc))

# inf.close()

var_up, var_lo, keys = initialize_variables()

501


In [7]:
# ## Initialize files
# depth_max=4.5
# prefix = 'areas'
# affixup='_norm_small0.8_up_%.1f.txt'%depth_max
# affixlo='_norm_small0.8_lo_%.1f.txt'%depth_max


affixup='%s%.1f_up_%.1f_c%.1f_v2.txt'%(affix, striangle, depth_max,cutoff)
affixlo='%s%.1f_lo_%.1f_c%.1f_v2.txt'%(affix, striangle, depth_max,cutoff)
filenames_up = np.array(['%s0%s'%(prefix, affixup), '%s1%s'%(prefix, affixup), '%s01%s'%(prefix, affixup)])
filenames_lo = np.array(['%s0%s'%(prefix, affixlo), '%s1%s'%(prefix, affixlo), '%s01%s'%(prefix, affixlo)])

# framecount= 0 
# replacement_line = 'n frames = '
# for infile in filenames_up:
#     inf = open(savedir+infile, 'w')
#     inf.write(replacement_line + ' %d\n'%framecount)
#     inf.close()
    
# for infile in filenames_lo:
#     inf = open(savedir+infile, 'w')
#     inf.write(replacement_line + ' %d\n'%framecount)
#     inf.close()

In [8]:
def check_mesh_quality_areas(vertices, triangles, hist_bins, minmax):
    xmin, xmax = minmax[0], minmax[1]
    ymin, ymax = minmax[2], minmax[3]
    newmesh_triangle = np.array([[0, 1, 2]])
        
#     areas_m1 = np.zeros(len(triangles[:,0]))
#     areas_m2 = np.zeros(len(triangles[:,0]))
    areas_m1 = np.array([])
    areas_m2 = np.array([])
    
    ix = np.array([], dtype=int)
    ix = np.append(ix, np.where(vertices[:,0] < (xmin-3)))
    ix = np.append(ix, np.where(vertices[:,0] > (xmax+3)))
    ix = np.append(ix, np.where(vertices[:,1] < (ymin-3)))
    ix = np.append(ix, np.where(vertices[:,1] > (ymax+3)))
    
    for t, tr in enumerate(triangles):
        boar = [t in ix for t in tr]
        
        if True in boar:
            pass
        else:
            areas_m1 = np.append(areas_m1, herons_formula(vertices[tr[0]], vertices[tr[1]], vertices[tr[2]]))
            new_vertices = np.array([])
            for i in tr:
                new_vertices = np.append(new_vertices, vertices[i])
            new_vertices = new_vertices.reshape(3, 3)
            mesh_new = o3d.geometry.TriangleMesh()
            mesh_new.vertices=o3d.utility.Vector3dVector(new_vertices[:, :3])
            mesh_new.triangles=o3d.utility.Vector3iVector(newmesh_triangle[:, :3])
#             areas_m2[t] = mesh_new.get_surface_area()
            areas_m2 = np.append(areas_m2, mesh_new.get_surface_area())
    
    bins = hist_bins
    hist_m1 = np.histogram(areas_m1, bins, density=True)
    hist_m2 = np.histogram(areas_m2, bins, density=True)
    binsreal = (bins[:-1]+bins[1:])/2
    return binsreal, hist_m1[0], hist_m2[0], areas_m1, areas_m2

def grid_plane_triangulated_m(grid_l1, size_triangle=1.0, num_it1=1, num_it2=1, method='cvt full', simp_cutoff=1.0):
    grid_l1.remove_non_manifold_edges()
    
    grid_l1.merge_close_vertices(simp_cutoff)
    grid_l1.remove_degenerate_triangles()
    grid_l1.remove_unreferenced_vertices()
    grid_l1.remove_duplicated_triangles()
    grid_l1.remove_duplicated_vertices()

    current_n_triangles = np.asarray(grid_l1.triangles).shape[0]

    n_triangles = int(grid_l1.get_surface_area()/size_triangle)

    versions_of_4 = int(np.ceil(( n_triangles/current_n_triangles )**(1/4)))
    n_triangles_first = int(n_triangles/ (4**versions_of_4))

    mesh_smp = grid_l1.simplify_quadric_decimation(
        target_number_of_triangles=n_triangles_first)
    
    mesh_smp.remove_non_manifold_edges()
    
    mesh_smp.merge_close_vertices(simp_cutoff)
    mesh_smp.remove_degenerate_triangles()
    mesh_smp.remove_unreferenced_vertices()
    mesh_smp.remove_duplicated_triangles()
    mesh_smp.remove_duplicated_vertices()

    vertices=mesh_smp.vertices
    triangles=mesh_smp.triangles
    
    ## Centroidal Voronoi Tessellation (CVT)
    #v1
    points, cells = optimesh.optimize_points_cells(vertices, triangles, method, 1.0e-9, num_it1)
#     points, cells = optimesh.optimize_points_cells(vertices, triangles, "cvt full", 1.0e-9, num_it1)
#     points, cells = optimesh.optimize_points_cells(vertices, triangles, "cvt (block-diagonal)", 1.0e-9, num_it1)

    mesh_new = o3d.geometry.TriangleMesh()
    mesh_new.vertices=o3d.utility.Vector3dVector(points[:, :3])
    mesh_new.triangles=o3d.utility.Vector3iVector(cells[:, :3])

    mesh_new.compute_triangle_normals()
    mesh_new.normalize_normals()

    grid_l1_new = o3d.geometry.TriangleMesh.subdivide_midpoint(mesh_new, versions_of_4)
    grid_l1_new.paint_uniform_color([0.9, 0.9, 0.9])

    vertices=grid_l1_new.vertices
    triangles=grid_l1_new.triangles
    
    ## Centroidal Voronoi Tessellation (CVT)
    #v2
    points, cells = optimesh.optimize_points_cells(vertices, triangles, method, 1.0e-9, num_it2)
#     points, cells = optimesh.optimize_points_cells(vertices, triangles, "cvt full", 1.0e-9, num_it2)
#     points, cells = optimesh.optimize_points_cells(vertices, triangles, "cvt (block-diagonal)", 1.0e-9, num_it2)
    

    mesh_new2 = o3d.geometry.TriangleMesh()
    mesh_new2.vertices=o3d.utility.Vector3dVector(points[:, :3])
    mesh_new2.triangles=o3d.utility.Vector3iVector(cells[:, :3])
    
    mesh_new2.compute_triangle_normals()
    mesh_new2.normalize_normals()

    return mesh_new2




In [9]:
def rms_average_roughness_x(vertices_lo, slivers, slivethickbess):
    rq = np.array([])
    for s in slivers:
        ix = np.where(vertices_lo[:,0] < s+2)
        check_vert = vertices_lo[ix]

        ix2 = np.where(check_vert[:,0] > s)
        check_v1 = check_vert[ix2]
        v1_ss = check_v1[check_v1[:, 1].argsort()]
        
        rq = np.append(rq, rq_calc(v1_ss[:,1], v1_ss[:,2]))
    return rq

def rms_average_roughness_y(vertices_lo, slivers, slivethickbess):
    rq = np.array([])
    for s in slivers:
        ix = np.where(vertices_lo[:,1] < s+2)
        check_vert = vertices_lo[ix]

        ix2 = np.where(check_vert[:,1] > s)
        check_v1 = check_vert[ix2]
        v1_ss = check_v1[check_v1[:, 0].argsort()]
        
        rq = np.append(rq, rq_calc(v1_ss[:,0], v1_ss[:,2]))
    return rq


def rq_calc(xx, yy):
    temp = 0
    yy = yy-np.mean(yy)
    for i in range(0, len(xx)-1):
        dx = (xx[i] + xx[i+1])/2
        dy = ((yy[i])**2 + (yy[i+1])**2)/2
        temp += dy*dx
    rq = np.sqrt((1/np.max(xx))*temp)
    return rq


In [10]:
def fill_in_defect_matrix7(gl2_leaflet, grid_coords, normals, box, depth_max, upper=False, tinygridradius=10, cutoff=0.6):
    m_polar = 1.0
    m_aliphatic = 0.001
    dist_clo = cutoff
    ## Initalize matrix with size of number of cells
    M_1 = np.ones(np.shape(grid_coords)[0])
    M_1[:] = np.nan
    upperval = 1
    if (upper == True):
        upperval = -1
        ## v2: added this: (v1, without this part belw.)
        # normals = normals * upperval
    
    ## Loop over all lipids in the leaflet
    for gl2 in gl2_leaflet[:]:
        res = gl2.residue # Get the residue
        gl2_coords = gl2.position # position of the GL2 in A
        res_coords = res.atoms.positions # all positions of all beads in the molecule in A
        
        ## Get index of the closest gridcell to the gl2_coordinate
        index_closest_cell = np.argmin((grid_coords[:,0] - gl2_coords[0])**2 + (grid_coords[:,1] - gl2_coords[1])**2 + (grid_coords[:,2] - gl2_coords[2])**2, axis=0 )
        
        ## Get the corresponding coordinates for the grid cell that is closest to the gl2 coordinate
        cell_coord = grid_coords[index_closest_cell] ## corresponding coordinate
        
        ## get averaged normal vector over small grid of radius 10A 
        normal = np.mean(normals[np.where(np.sum((grid_coords - gl2_coords)**2, axis=1) < tinygridradius**2)], axis=0)
        if np.isnan(normal[0]):
            print(normal, res, index_closest_cell)
            normal = normals[index_closest_cell]
            
        ## Take an average of normal vectors
        avnormal = upperval * normal
        avnormal = avnormal / np.sqrt(np.sum(avnormal**2)) ## Normalize
        
        ## Move grid such that is goes through the GL2 bead of the lipid
        grid_coords_m = grid_coords + (gl2_coords-cell_coord)*avnormal
                
        ## Move the grid such that is is 1.0 lower than the GL2.
        if (upper == True):
            grid_coords_m = grid_coords_m - depth_max*avnormal ## Upper
        else:
            grid_coords_m = grid_coords_m + depth_max*avnormal ## Lower
            
        ## Get closest index of moved grid:
        index_closest_cell2 = np.argmin((grid_coords_m[:,0] - gl2_coords[0])**2 + (grid_coords_m[:,1] - gl2_coords[1])**2 + (grid_coords_m[:,2] - gl2_coords[2])**2, axis=0 )
        cell_coord2 = grid_coords_m[index_closest_cell2] ## corresponding cell
        
        ## Move positions of beads in molecule such that cell_coords2 = [0, 0, 0], origin
#             dr_atom_ref = min_image_dist(res_coords, cell_coord2, box)
        dr_atom_c = res_coords-cell_coord2
        
        ## Flip normal (currently pointed into the membrane) after flipping point out!
        normal_gl2 = normal ##[index_closest_cell]
        
        ## Calculate the inner product. If the inner product is below zero >> The bead is below the grid 1.0 below gl2
        cos_alpha = np.inner(dr_atom_c, normal_gl2) ## If above 0 >> below surface
            
        for a, atom in enumerate(res.atoms):  #[np.where(cos_alpha >= 0)]:
            atom_name = atom.name ## get atom name to determine whether it is aliphatic or not.
            
            atom_radius, m_value = get_radius_and_defect_type(atom_name, cos_alpha[a])
            
            dist_lim = (dist_clo+atom_radius)**2

            ## Move lipid to grid:
            ix_closest = np.argmin((grid_coords[:,0] - atom.position[0])**2 + (grid_coords[:,1] - atom.position[1])**2 + (grid_coords[:,2] - atom.position[2])**2, axis=0 )
#             dist = np.sqrt(np.sum((atom.position-grid_coords[ix_closest])**2))
#             minorplus = np.linalg.norm(atom.position) - np.linalg.norm(grid_coords[ix_closest])
#             dist = (minorplus / abs(minorplus)) * dist
            dist = np.dot((atom.position-grid_coords[ix_closest]), avnormal)
        
            pos = atom.position - (dist)*avnormal
            delta_x = np.sum((grid_coords-pos)**2, axis=1)
            ix_lim = np.unique(np.where(delta_x <= dist_lim)[0])
                
            for x1 in ix_lim:
                if np.isnan(M_1[x1]):
                    if m_value == m_polar:
                        M_1[x1] = 1
                    else:   
                        M_1[x1] = 0
            M_1[ix_lim] += m_value
    return M_1

def get_cutoff(grid_coords, n_neighbours, pos, n_for_cutoff):
    dist = average_distance_neighbours(grid_coords, n_neighbours)
    ix_not = np.where(grid_coords[:,0] < np.min(pos[:,0])-3)
    ix_not = np.append(ix_not, np.where(grid_coords[:,0] > np.max(pos[:,0])+3))
    ix_not = np.append(ix_not, np.where(grid_coords[:,1] < np.min(pos[:,1])-3))
    ix_not = np.append(ix_not, np.where(grid_coords[:,1] > np.max(pos[:,1])+3))
    dist = np.delete(dist, np.unique(ix_not), axis=0)
    cutoff = np.mean(np.mean(dist[:,-n_for_cutoff:], axis=1))/2
    return cutoff

In [11]:
print(savedir)
colors = ['#001219', '#005F73', '#0A9396', '#94D2BD', '#E9D8A6', '#EE9B00', '#CA6702', '#BB3E03', '#AE2012', '#9B2226']
print(affix)

/Users/ankush/Desktop/Ankush/C60-Chol/Defects/flat-325K-membrane-only/more-radius/
_dppc303


In [12]:
## Grid fitting:
striangle=1.0
cutoff = 0.9
numit1 = 6
numit2 = 5

methods = ['lloyd', "cvt full", "cvt (block-diagonal)", "cpt-linear-solve", "cpt-fixed-point", "cpt-quasi-newton", "odt-fixed-point"]
mm = methods[2]
depth_max_lo = 4.5

## Imaging
imaging = True
everyframe=1
edge=5
pixel=1.2

## Trajectory
skip_arr = np.array([])

## Defect matrix
depth_max = depth_max_lo

inf = open(savedir+'settings%s%.1f_c%.1f_new_cutoffnew_newrange.txt'%(affix,striangle,cutoff), 'w')

inf.write('size triangle: %f \n'%striangle)
inf.write('number of iterations 1: %d  & 2: %d \n'%(numit1, numit2))
inf.write('depth_max: %f \n'%depth_max)
inf.write('savedir: %s \n'%savedir)
inf.write('inputpath: %s \n'%workdir)
inf.write('inputfile: %s & %s \n'%(infiletpr, infilextc))

inf.close()

var_up, var_lo, keys = initialize_variables()
# 
affixup='%s%.1f_up_%.1f_c%.1f_new_cutoffnew_newrange2.txt'%(affix, striangle, depth_max,cutoff)
affixlo='%s%.1f_lo_%.1f_c%.1f_new_cutoffnew_newrange2.txt'%(affix, striangle, depth_max,cutoff)
filenames_up = np.array(['%s0%s'%(prefix, affixup), '%s1%s'%(prefix, affixup), '%s01%s'%(prefix, affixup)])
filenames_lo = np.array(['%s0%s'%(prefix, affixlo), '%s1%s'%(prefix, affixlo), '%s01%s'%(prefix, affixlo)])

framecount= 0 
replacement_line = 'n frames = '
for infile in filenames_up:
    inf = open(savedir+infile, 'w')
    inf.write(replacement_line + ' %d\n'%framecount)
    inf.close()
    
for infile in filenames_lo:
    inf = open(savedir+infile, 'w')
    inf.write(replacement_line + ' %d\n'%framecount)
    inf.close()

## FLAT
for infile in filenames_up:
    inf = open(savedir+'r1_'+infile, 'w')
    inf.write(replacement_line + ' %d\n'%framecount)
    inf.close()
    
for infile in filenames_lo:
    inf = open(savedir+'r1_'+infile, 'w')
    inf.write(replacement_line + ' %d\n'%framecount)
    inf.close()

## CURVE 1
for infile in filenames_up:
    inf = open(savedir+'r2_'+infile, 'w')
    inf.write(replacement_line + ' %d\n'%framecount)
    inf.close()
    
for infile in filenames_lo:
    inf = open(savedir+'r2_'+infile, 'w')
    inf.write(replacement_line + ' %d\n'%framecount)
    inf.close()

## CURVE 2
for infile in filenames_up:
    inf = open(savedir+'r3_'+infile, 'w')
    inf.write(replacement_line + ' %d\n'%framecount)
    inf.close()
    
for infile in filenames_lo:
    inf = open(savedir+'r3_'+infile, 'w')
    inf.write(replacement_line + ' %d\n'%framecount)
    inf.close()

In [13]:
def rotate_normals(pcd, rotation_mat, flip_norms):
    pcd_trans = o3d.geometry.PointCloud()
    pcd_trans.points = pcd.points

    normals = np.asarray(pcd.normals)
    if flip_norms == True:
        normals = normals * -1.0
    normals_rotated = np.dot(normals, rotation_mat.T)
    pcd_trans.normals = o3d.utility.Vector3dVector(normals_rotated)
    return pcd_trans 

def get_regions_2(pcd, grid_coords, vertices, triangles, binsize=20, continuous_in_dir=1, coscut = 0.835):
    uppoints = np.asarray(pcd.points)
    normals_up_rotated = np.asarray(pcd.normals)
    points_new = np.array([])
    norms_new = np.array([])

    binedge_r = np.min(uppoints[:,continuous_in_dir])
    binedge_l = binsize + binedge_r
    
    ix = np.logical_and(uppoints[:,continuous_in_dir] >= binedge_r, uppoints[:,continuous_in_dir] < binedge_l)
    points_new = np.append(points_new, np.mean(uppoints[ix], axis=0))
    norms_new = np.append(norms_new, np.mean(normals_up_rotated[ix], axis=0))

    translate_vec = binsize*(np.mean(normals_up_rotated[ix], axis=0)/np.linalg.norm(np.mean(normals_up_rotated[ix], axis=0)))
    binedge_r = binedge_l
    trans_points = np.mean(uppoints[ix], axis=0) + (translate_vec)
    binedge_l = trans_points[continuous_in_dir]

    while binedge_l < np.max(uppoints[:,continuous_in_dir]):
        ix = np.logical_and(uppoints[:,continuous_in_dir] >= binedge_r, uppoints[:,continuous_in_dir] < binedge_l)
        points_new = np.append(points_new, np.mean(uppoints[ix], axis=0))
        norms_new = np.append(norms_new, np.mean(normals_up_rotated[ix], axis=0))

        translate_vec = binsize*(np.mean(normals_up_rotated[ix], axis=0)/np.linalg.norm(np.mean(normals_up_rotated[ix], axis=0)))
        binedge_r = binedge_l
        trans_points = np.mean(uppoints[ix], axis=0) + translate_vec
        binedge_l = trans_points[continuous_in_dir]

    # print(points_new.shape)

    # print(points_new.shape, points_new.reshape(int(len(points_new)/3), 3))
    points_new2 = points_new.reshape(int(len(points_new)/3), 3)
    norms_new2 = norms_new.reshape(int(len(norms_new)/3), 3)
    pcd_new = o3d.geometry.PointCloud()
    pcd_new.points = o3d.utility.Vector3dVector(points_new2)
    pcd_new.normals = o3d.utility.Vector3dVector(norms_new2)

    # o3d.visualization.draw_geometries([pcd_new], point_show_normal=True)

    cosalpha_all = np.zeros((len(points_new2)))
    # sinalpha_all = np.zeros((len(points_new2)))

    valmin = 2
    val = 2
    for pp in range(0, len(points_new2)):
        if (pp + val) >= len(points_new2):
            val2 = abs(len(points_new2)-(pp+val))
        else:
            val2 = pp + val
        if (pp - valmin ) < 0:
            val1 = len(points_new2) + (pp-valmin)
        else:   
            val1 = pp - valmin
        cosalpha_all[pp] = np.dot(norms_new2[val1], norms_new2[val2]) # - np.dot(norms_new2[pp], norms_new2[pp]) 
        # sinalpha_all[pp] = np.linalg.norm(np.cross(norms_new2[val1], norms_new2[val2])) 

    # sinalpha_all[0] =sinalpha_all[1]
    # sinalpha_all[-1]=sinalpha_all[-2]

    cosalpha_all[0] =cosalpha_all[1]
    cosalpha_all[-1]=cosalpha_all[-2]

    # coscut = 0.835
    
    # print(cosalpha_all)

    ix_bool = cosalpha_all <= coscut
    startbool = ix_bool[0]
    ybins = np.array([])

    for isb, sb in enumerate(ix_bool):
        if sb != startbool:
            ybins = np.append(ybins, points_new2[:,continuous_in_dir][isb])
            startbool = sb

    ybins_res = ybins.reshape(int(len(ybins)/2), 2)
    print('ybins')
    print(ybins_res)
    for yi, yb in enumerate(ybins_res):
        if yi == 0:
            ixs_f1 = (grid_coords[:,continuous_in_dir] <= yb[0])
            ixs_f2 = np.logical_and(grid_coords[:,continuous_in_dir] > yb[0], grid_coords[:,continuous_in_dir] < yb[1])
            saveyi = yb[1]
            # print(np.where(ixs_f1==False)[0].shape, np.where(ixs_f2==False)[0].shape)
        elif yi == (len(ybins_res)-1):
            ixs_f1_t = np.logical_and(grid_coords[:,continuous_in_dir] >= saveyi, grid_coords[:,continuous_in_dir] <= yb[0])
            ixs_f1 = np.logical_or(ixs_f1_t, ixs_f1)

            ixs_f2_t = np.logical_and(grid_coords[:,continuous_in_dir] > yb[0], grid_coords[:,continuous_in_dir] < yb[1])
            ixs_f2 = np.logical_or(ixs_f2_t, ixs_f2)
        
            ixs_f1 = np.logical_or(ixs_f1, grid_coords[:,continuous_in_dir] >= yb[1])
            # ixs_f1 = np.logical_and(ixs_f1_t, ixs_f1)
        else:
            ixs_f1_t = np.logical_and(grid_coords[:,continuous_in_dir] >= saveyi, grid_coords[:,continuous_in_dir] <= yb[0])
            ixs_f1 = np.logical_or(ixs_f1_t, ixs_f1)

            ixs_f2_t = np.logical_and(grid_coords[:,continuous_in_dir] > yb[0], grid_coords[:,continuous_in_dir] < yb[1])
            ixs_f2 = np.logical_or(ixs_f2_t, ixs_f2)

            saveyi = yb[1]

    mesh_f1 = o3d.geometry.TriangleMesh()
    mesh_f2 = o3d.geometry.TriangleMesh()
    mesh_f1.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_f2.vertices = o3d.utility.Vector3dVector(vertices)

    mesh_f1.triangles = o3d.utility.Vector3iVector(triangles[ixs_f1])
    mesh_f2.triangles = o3d.utility.Vector3iVector(triangles[ixs_f2])

    mesh_f1.paint_uniform_color([0.9, 0.2, 0.9])
    mesh_f2.paint_uniform_color([0.9, 0.9, 0.9])
    return ixs_f1, ixs_f2, mesh_f1, mesh_f2, pcd_new

rotation_matrix_x = np.array([[1, 0, 0],
                            [0, 0, -1],
                            [0, 1, 0]])

def get_ranges_pos_neg_flat(triang_normals, minmax, plane_direction, grid_coords, buffer=30, dx=10, rangepos=[5, 50], rangeneg=[250, 355], view_plot=False):
    # plane_direction = 'xz_noty'
    xyz_label=np.array(['x', 'y', 'z'])

    minbuf = 5
    xval = np.where(xyz_label == plane_direction[0])[0][0]
    yval = np.where(xyz_label == plane_direction[1])[0][0]
    zval = np.where(xyz_label == plane_direction[-1])[0][0]

    grid_coords_use = np.logical_and(grid_coords[:,1] > (minmax[2]+minbuf), grid_coords[:,1] < (minmax[3] - minbuf))

    xmin, xmax = np.min(grid_coords[grid_coords_use,xval]), np.max(grid_coords[grid_coords_use,xval])
    ymin, ymax = np.min(grid_coords[grid_coords_use,yval]), np.max(grid_coords[grid_coords_use,yval])

    xbins = np.arange(int(xmin)-buffer, int(xmax)+buffer, dx)
    ybins = np.arange(int(ymin)-buffer, int(ymax)+buffer, dx)

    voxelgrid_1 = np.zeros_like(grid_coords[:,xval])
    voxelgrid_2 = np.zeros_like(grid_coords[:,xval])

    for xxb in xbins:
        for yyb in ybins:
            grid_coords_use_ixx = np.logical_and(grid_coords[:,xval] > (xxb-(dx/2)), grid_coords[:,xval] <= (xxb+(dx/2)))
            grid_coords_use_ixy = np.logical_and(grid_coords[:,yval] > (yyb-(dx/2)), grid_coords[:,yval] <= (yyb+(dx/2)))
            grid_coords_use_ix  = np.logical_and(grid_coords_use_ixx, grid_coords_use_ixy)

            if True in grid_coords_use_ix:
                ## do something
                # print(ix, iy)

                avnormal = np.mean(triang_normals[grid_coords_use_ix], axis=0)
                centroid = np.mean(grid_coords[grid_coords_use_ix], axis=0)
                if zval == 1:
                    normal_forward = rotate_vector_aroundy(avnormal, 270)
                    normal_backward = rotate_vector_aroundy(avnormal, 90)
                if zval == 0:
                    normal_forward = rotate_vector_aroundx(avnormal, 270)
                    normal_backward = rotate_vector_aroundx(avnormal, 90)
                if zval == 2:
                    normal_forward = rotate_vector_aroundz(avnormal, 270)
                    normal_backward = rotate_vector_aroundz(avnormal, 90)

                centroid_forward = centroid + dx*normal_forward
                centroid_backward = centroid + dx*normal_backward

                grid_coords_use_ixx_f = np.logical_and(grid_coords[:,xval] > (centroid_forward[xval]-(dx/2)), grid_coords[:,xval] <= (centroid_forward[xval]+(dx/2)))
                grid_coords_use_ixy_f = np.logical_and(grid_coords[:,yval] > (centroid_forward[yval]-(dx/2)), grid_coords[:,yval] <= (centroid_forward[yval]+(dx/2)))
                grid_coords_use_ix_f  = np.logical_and(grid_coords_use_ixx_f, grid_coords_use_ixy_f)
                grid_coords_use_ixx_b = np.logical_and(grid_coords[:,xval] > (centroid_backward[xval]-(dx/2)), grid_coords[:,xval] <= (centroid_backward[xval]+(dx/2)))
                grid_coords_use_ixy_b = np.logical_and(grid_coords[:,yval] > (centroid_backward[yval]-(dx/2)), grid_coords[:,yval] <= (centroid_backward[yval]+(dx/2)))
                grid_coords_use_ix_b  = np.logical_and(grid_coords_use_ixx_b, grid_coords_use_ixy_b)
            
                normal_forward = np.mean(triang_normals[grid_coords_use_ix_f], axis=0)
                normal_backward = np.mean(triang_normals[grid_coords_use_ix_b], axis=0)

                angle_1 = calculate_angle(normalize_vector(normal_backward), normalize_vector(avnormal), xval, yval)
                angle_2 = calculate_angle(normalize_vector(avnormal), normalize_vector(normal_forward), xval, yval)

                voxelgrid_1[grid_coords_use_ix] = angle_1
                voxelgrid_2[grid_coords_use_ix] = angle_2

            else:
                pass
    
    voxelav = (voxelgrid_1+voxelgrid_2)/2

    if view_plot:
        plt.plot(voxelav, marker='o', ls='None')
        plt.plot(voxelgrid_1, marker='.', ls='None')
        plt.plot(voxelgrid_2, marker='.', ls='None')
        plt.show()
    ## range pos = [5 to 50]
    ix_rangepos = np.logical_and(voxelav >= rangepos[0], voxelav < rangepos[1])

    ## rangeneg = [250 to 355]
    ix_rangeneg = np.logical_and(voxelav >= rangeneg[0], voxelav < rangeneg[1])

    ix_rangeflat = np.logical_not(np.logical_or(ix_rangepos, ix_rangeneg))

    return ix_rangepos, ix_rangeneg, ix_rangeflat


def calculate_angle(vector1, vector2, xval, yval):
    dot_product = sum(a * b for a, b in zip(vector1, vector2))
    magnitude1 = math.sqrt(sum(a * a for a in vector1))
    magnitude2 = math.sqrt(sum(a * a for a in vector2))
    cos_theta = dot_product / (magnitude1 * magnitude2)

    # Ensure the cosine value is within the valid range [-1, 1]
    cos_theta = max(min(cos_theta, 1), -1)

    # Calculate the angle in radians
    theta_rad = math.acos(cos_theta)

    # Convert radians to degrees
    theta_deg = math.degrees(theta_rad)

    # Ensure the angle is within the range [0, 360]
    if vector1[xval]*vector2[yval] - vector1[yval]*vector2[xval] > 0:
        theta_deg = 360 - theta_deg

    return theta_deg

def normalize_vector(vector):
    magnitude = math.sqrt(sum(a * a for a in vector))
    normalized = [a / magnitude for a in vector]
    return normalized

def rotate_vector_aroundx(vector, angle_deg):
    angle_rad = math.radians(angle_deg)
    x = vector[0]
    y = vector[1] * math.cos(angle_rad) - vector[2] * math.sin(angle_rad)
    z = vector[1] * math.sin(angle_rad) + vector[2] * math.cos(angle_rad)
    return np.array([x, y, z])

def rotate_vector_aroundy(vector, angle_deg):
    angle_rad = math.radians(angle_deg)
    x = vector[0] * math.cos(angle_rad) + vector[2] * math.sin(angle_rad)
    y = vector[1]
    z = -vector[0] * math.sin(angle_rad) + vector[2] * math.cos(angle_rad)
    return np.array([x, y, z])

def rotate_vector_aroundz(vector, angle_deg):
    angle_rad = math.radians(angle_deg)
    x = vector[0] * math.cos(angle_rad) - vector[1] * math.sin(angle_rad)
    y = vector[0] * math.sin(angle_rad) + vector[1] * math.cos(angle_rad)
    z = vector[2]
    return np.array([x, y, z])



def make_grid_flat_test3(gro, ag, lower=True, radius_norms=100, max_nn=30, seltail={'DPPC': 'name C4*'}): 
    positions = ag.positions
    ## Make triangle mesh from pointcloud:
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(positions[:,:3])

    ## Compute normals of the points:
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_norms, max_nn=max_nn))
    # pcd.orient_normals_consistent_tangent_plane(100) ## THIS GIVES CORRECT NORMAL VECTORS!  ## DOES NOT WORK FOR THIS DONT KNOW WHY
    pcd.normalize_normals()

    # normals = np.asarray(pcd.normals)
    
    resag = ag.resids ## Get resids of the point cloud beads
    resnames = ag.resnames
    tails = np.array([])
    for ir, ra in enumerate(resag):
                resname_for_tail = resnames[ir]
                tails = np.append(tails, gro.select_atoms('(resid %d and (%s))'%(ra, seltail[resname_for_tail])).positions)

    tails = tails.reshape((int(len(tails)/3), 3))
    tails_av = (tails[0::2] + tails[1::2])/2 ## get an average of the tail beads, thus 1 bead per tail
    dir_norm = (tails_av - positions)/ np.vstack((np.linalg.norm((tails_av - positions), axis=1), np.linalg.norm((tails_av - positions), axis=1), np.linalg.norm((tails_av - positions), axis=1))).T

    normals_3d = np.asarray(pcd.normals)
    dots = np.array([])
    for i in range(0, len(dir_norm)):
                dots = np.append(dots, np.dot(dir_norm[i], normals_3d[i]) / (np.linalg.norm(dir_norm[i])*np.linalg.norm(normals_3d[i])))
    ix_normals_to_flip = np.where(dots < 0)
    normals_3d[ix_normals_to_flip] = -1 * normals_3d[ix_normals_to_flip]
    pcd.normals = o3d.utility.Vector3dVector(normals_3d)

    # if lower==True:
    #     normals = -1*np.asarray(pcd.normals)
    #     pcd.normals = o3d.utility.Vector3dVector(normals)
    
    ## Make surface with poisson: 
    mesh_pois, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)
    
    with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
        triangle_clusters, cluster_n_triangles, cluster_area = (
            mesh_pois.cluster_connected_triangles())
    triangle_clusters = np.asarray(triangle_clusters)
    cluster_n_triangles = np.asarray(cluster_n_triangles)
    cluster_area = np.asarray(cluster_area)
    
    triangles_to_remove = cluster_n_triangles[triangle_clusters] < 100
    mesh_pois.remove_triangles_by_mask(triangles_to_remove)
    
    # Check if mesh is continuous, no manifold edges, watertight: 
    mesh_pois.remove_non_manifold_edges()
    mesh_pois.remove_duplicated_vertices()
    mesh_pois.remove_duplicated_triangles()
    
    mesh_pois.compute_triangle_normals()
    mesh_pois.normalize_normals()
    return mesh_pois, pcd

print(len(gro.trajectory))

501


In [14]:
# cutoff_lo_arr = np.array([])
# cutoff_up_arr = np.array([])
# frame_number = np.array([])
everyframe=20

seltail= {'DPPC': 'name C4*',
          'DPPG': 'name C4*',
          'CHOL': 'name C*'}

rangepos1 = [5, 30]
rangeneg1 = [305, 354]
simp_cutoff = 2.0
simp_cutoff2 = 3.0

print(gro.trajectory)

for ts in gro.trajectory[1:]:
    start_time = time.time()
    print(ts.frame)
    
    upper, lower = get_selection_phaseseparated(gro, 15, 12) ## Make selection of beads
    
    # # uppos = add_pbc_egdes(upper.positions, ts.dimensions[:3], edge_val = 1) ## add pbc of beads for fitting of surface
    # # lopos = add_pbc_egdes(lower.positions, ts.dimensions[:3], edge_val = 1) ## add pbc of beads for fitting of surface

    # # grid_up, pcd_up = make_grid_flat_test2(uppos, lower=True, radius_norms=100, max_nn=30)  ## Make a surface of upper leaflet
    # # grid_lo, pcd_lo = make_grid_flat_test2(lopos, lower=False, radius_norms=100, max_nn=30)  ## Make a surface of lower leaflet

    # grid_up, pcd_up = make_grid_flat_test2(upper.positions, lower=True, radius_norms=100, max_nn=30)  ## Make a surface of upper leaflet
    # grid_lo, pcd_lo = make_grid_flat_test2(lower.positions, lower=False, radius_norms=100, max_nn=30)  ## Make a surface of lower leaflet
    # grid_up.merge_close_vertices(simp_cutoff2)
    # grid_lo.merge_close_vertices(simp_cutoff2)

    grid_up, pcd_up = make_grid_flat_test3(gro, upper, lower=True, radius_norms=100, max_nn=30, seltail=seltail)   ## Make a surface of upper leaflet
    grid_lo, pcd_lo = make_grid_flat_test3(gro, lower, lower=False, radius_norms=100, max_nn=30, seltail=seltail)  ## Make a surface of lower leaflet

    # print('remesh grid')
    regrid_up = grid_plane_triangulated_m(grid_up, size_triangle=striangle, num_it1=numit1, num_it2=numit2, method=mm, simp_cutoff=simp_cutoff)  ## Remesh grid
    regrid_lo = grid_plane_triangulated_m(grid_lo, size_triangle=striangle, num_it1=numit1, num_it2=numit2, method=mm, simp_cutoff=simp_cutoff)  ## Remesh grid
    
    vertices_up = np.asarray(regrid_up.vertices)
    triangles_up = np.asarray(regrid_up.triangles)
    grid_coords_up = get_centroid_of_triangles(vertices_up, triangles_up)
    normals = np.asarray(regrid_up.triangle_normals)
    gl2_leaflet = upper
    
    cutoff_up = get_cutoff(grid_coords_up, 12, upper.positions, 3)
    
    M_1 = fill_in_defect_matrix7(gl2_leaflet, grid_coords_up, normals, box, depth_max_lo, upper=True, tinygridradius=5, cutoff=cutoff_up)
    M_discrete_1_up = np.where(M_1 >= 1, 2, np.where(M_1 > 0, 1, np.where(M_1 >-0.5, 0, -1)))
    
    minmax = np.array([np.min(gl2_leaflet.positions[:,0]), np.max(gl2_leaflet.positions[:,0]), np.min(gl2_leaflet.positions[:,1]), np.max(gl2_leaflet.positions[:,1])])
    areas0, areas1, areas01, mesh_0up6, mesh_1up6, mesh_01up6 = get_clusters_using_mesh_stitch(M_discrete_1_up, vertices_up, triangles_up, minmax)
    

    # # var_up = put_in_variables(var_up, keys, areas01, mesh_01up6, grid_coords_up, regrid_up, box, upper.positions)
    
    write_file(areas0, savedir+filenames_up[0], replacement_line)
    write_file(areas1, savedir+filenames_up[1], replacement_line)
    write_file(areas01, savedir+filenames_up[2], replacement_line)

    ## GET REGIONS
    # pcd_trans_up = rotate_normals(pcd_up, rotation_matrix_x, flip_norms=False)
    # ixs_f1_up, ixs_f2_up, mesh_f1_up, mesh_f2_up, pcd_new_up = get_regions_2(pcd_trans_up, grid_coords_up, vertices_up, triangles_up, binsize=20, continuous_in_dir=1, coscut = 0.835)
    # ix_curve_up, ix_curve_lo, ix_flat_1 = get_curve_sel(upper, 'y', 'name GL2', 0.01, -0.01, 100, grid_coords_up)
    ixr1, ixr2, ixr3 = get_ranges_pos_neg_flat(np.asarray(regrid_up.triangle_normals), minmax, 'yz_notx', grid_coords_up, buffer=30, dx=10, rangepos=[4, 50], rangeneg=[250, 356], view_plot=False)
    
    areas0, areas1, areas01, mesh_0lo6_f1, mesh_1lo6_f1, mesh_01lo6_f1 = get_clusters_using_mesh_stitch(M_discrete_1_up[ixr1], vertices_up, triangles_up[ixr1], minmax)
    write_file(areas0, savedir+'r1_'+filenames_up[0], replacement_line)
    write_file(areas1, savedir+'r1_'+filenames_up[1], replacement_line)
    write_file(areas01, savedir+'r1_'+filenames_up[2], replacement_line)

    areas0, areas1, areas01, mesh_0lo6_c2, mesh_1lo6_c2, mesh_01lo6_c2 = get_clusters_using_mesh_stitch(M_discrete_1_up[ixr2], vertices_up, triangles_up[ixr2], minmax)
    write_file(areas0, savedir+'r2_'+filenames_up[0], replacement_line)
    write_file(areas1, savedir+'r2_'+filenames_up[1], replacement_line)
    write_file(areas01, savedir+'r2_'+filenames_up[2], replacement_line)

    areas0, areas1, areas01, mesh_0lo6_c1, mesh_1lo6_c1, mesh_01lo6_c1 = get_clusters_using_mesh_stitch(M_discrete_1_up[ixr3], vertices_up, triangles_up[ixr3], minmax)
    write_file(areas0, savedir+'r3_'+filenames_up[0], replacement_line)
    write_file(areas1, savedir+'r3_'+filenames_up[1], replacement_line)
    write_file(areas01, savedir+'r3_'+filenames_up[2], replacement_line)


#     print('lower')
    vertices_lo = np.asarray(regrid_lo.vertices)
    triangles_lo = np.asarray(regrid_lo.triangles)
    grid_coords_lo = get_centroid_of_triangles(vertices_lo, triangles_lo)
    normals = np.asarray(regrid_lo.triangle_normals)
    gl2_leaflet = lower
    
    cutoff_lo = get_cutoff(grid_coords_lo, 12, lower.positions, 3)

    M_1 = fill_in_defect_matrix7(gl2_leaflet, grid_coords_lo, normals, box, depth_max_lo, upper=False, tinygridradius=5, cutoff=cutoff_lo)
    M_discrete_1_lo = np.where(M_1 >= 1, 2, np.where(M_1 > 0, 1, np.where(M_1 >-0.5, 0, -1)))
    
    minmax = np.array([np.min(gl2_leaflet.positions[:,0]), np.max(gl2_leaflet.positions[:,0]), np.min(gl2_leaflet.positions[:,1]), np.max(gl2_leaflet.positions[:,1])])
    areas0, areas1, areas01, mesh_0lo6, mesh_1lo6, mesh_01lo6 = get_clusters_using_mesh_stitch(M_discrete_1_lo, vertices_lo, triangles_lo, minmax)
    
    # var_lo = put_in_variables(var_lo, keys, areas01, mesh_01lo6, grid_coords_lo, regrid_lo, box, lower.positions)
    
    write_file(areas0, savedir+filenames_lo[0], replacement_line)
    write_file(areas1, savedir+filenames_lo[1], replacement_line)
    write_file(areas01, savedir+filenames_lo[2], replacement_line)

#     pcd_trans_lo = rotate_normals(pcd_lo, rotation_matrix_x, flip_norms=True)
#     ixs_f1_lo, ixs_f2_lo, mesh_f1_lo, mesh_f2_lo, pcd_new_lo = get_regions_2(pcd_trans_lo, grid_coords_lo, vertices_lo, triangles_lo, binsize=20, continuous_in_dir=1, coscut = 0.835)
#     areas0_f1, areas1_f1, areas01_f1, mesh_0lo6_f1, mesh_1lo6_f1, mesh_01lo6_f1 = get_clusters_using_mesh_stitch(M_discrete_1_lo[ixs_f1_lo], vertices_lo, triangles_lo[ixs_f1_lo], minmax)
#     areas0_f2, areas1_f2, areas01_f2, mesh_0lo6_f2, mesh_1lo6_f2, mesh_01lo6_f2 = get_clusters_using_mesh_stitch(M_discrete_1_lo[ixs_f2_lo], vertices_lo, triangles_lo[ixs_f2_lo], minmax)

    # ix_curve_up, ix_curve_lo, ix_flat_1 = get_curve_sel(lower, 'y', 'name GL2', 0.01, -0.01, 100, grid_coords_lo)
    ixr1, ixr2, ixr3 = get_ranges_pos_neg_flat(np.asarray(regrid_lo.triangle_normals), minmax, 'yz_notx', grid_coords_lo, buffer=30, dx=10, rangepos=[4, 50], rangeneg=[250, 356], view_plot=False)
    
    areas0, areas1, areas01, mesh_0lo6_f1, mesh_1lo6_f1, mesh_01lo6_f1 = get_clusters_using_mesh_stitch(M_discrete_1_lo[ixr1], vertices_lo, triangles_lo[ixr1], minmax)
    write_file(areas0, savedir+'r1_'+filenames_lo[0], replacement_line)
    write_file(areas1, savedir+'r1_'+filenames_lo[1], replacement_line)
    write_file(areas01, savedir+'r1_'+filenames_lo[2], replacement_line)

    areas0, areas1, areas01, mesh_0lo6_c2, mesh_1lo6_c2, mesh_01lo6_c2 = get_clusters_using_mesh_stitch(M_discrete_1_lo[ixr2], vertices_lo, triangles_lo[ixr2], minmax)
    write_file(areas0, savedir+'r2_'+filenames_lo[0], replacement_line)
    write_file(areas1, savedir+'r2_'+filenames_lo[1], replacement_line)
    write_file(areas01, savedir+'r2_'+filenames_lo[2], replacement_line)

    areas0, areas1, areas01, mesh_0lo6_c1, mesh_1lo6_c1, mesh_01lo6_c1 = get_clusters_using_mesh_stitch(M_discrete_1_lo[ixr3], vertices_lo, triangles_lo[ixr3], minmax)
    write_file(areas0, savedir+'r3_'+filenames_lo[0], replacement_line)
    write_file(areas1, savedir+'r3_'+filenames_lo[1], replacement_line)
    write_file(areas01, savedir+'r3_'+filenames_lo[2], replacement_line)
    
    
    end_time = time.time() - start_time
    print(end_time/60)
    

<XTCReader /Users/ankush/Desktop/Ankush/C60-Chol/Defects/flat-325K-membrane-only/more-radius/DPPC.xtc with 501 frames of 39432 atoms>
1
[Open3D DEBUG] [ClusterConnectedTriangles] Compute triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done computing triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done clustering, #clusters=1
[Open3D DEBUG] [ClusterConnectedTriangles] Compute triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done computing triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done clustering, #clusters=1


/Users/ankush/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/ankush/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


[nan nan nan] <Residue DPPC, 237> 58050


/Users/ankush/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/ankush/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


2.199623000621796
2
[Open3D DEBUG] [ClusterConnectedTriangles] Compute triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done computing triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done clustering, #clusters=1
[Open3D DEBUG] [ClusterConnectedTriangles] Compute triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done computing triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done clustering, #clusters=1


...

[nan nan nan] <Residue DPPC, 937> 8408
[nan nan nan] <Residue DPPC, 1439> 59043


/Users/ankush/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/ankush/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


2.3526127696037293
500
[Open3D DEBUG] [ClusterConnectedTriangles] Compute triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done computing triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done clustering, #clusters=1
[Open3D DEBUG] [ClusterConnectedTriangles] Compute triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done computing triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done clustering, #clusters=1


/Users/ankush/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/ankush/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/Users/ankush/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/ankush/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


2.334703516960144


In [126]:

o3d.visualization.draw_geometries([regrid_up], mesh_show_wireframe=True, mesh_show_back_face=True)


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [127]:
## check the range!! to divide the buckles
ixr1, ixr2, ixr3 = get_ranges_pos_neg_flat(np.asarray(regrid_up.triangle_normals), minmax, 'yz_notx', grid_coords_up, buffer=30, dx=10, rangepos=[4, 50], rangeneg=[250, 356], view_plot=False)

triangles = np.asarray(regrid_up.triangles)
vertices = np.asarray(regrid_up.vertices)

mesh_r1 = o3d.geometry.TriangleMesh()
mesh_r2 = o3d.geometry.TriangleMesh()
mesh_r3 = o3d.geometry.TriangleMesh()

mesh_r1.vertices = o3d.utility.Vector3dVector(vertices)
mesh_r2.vertices = o3d.utility.Vector3dVector(vertices)
mesh_r3.vertices = o3d.utility.Vector3dVector(vertices)

mesh_r1.triangles = o3d.utility.Vector3iVector(triangles[ixr1])
mesh_r2.triangles = o3d.utility.Vector3iVector(triangles[ixr2])
mesh_r3.triangles = o3d.utility.Vector3iVector(triangles[ixr3])

mesh_r1.paint_uniform_color([0.9, 0.2, 0.9]) ## paars
mesh_r2.paint_uniform_color([0.9, 0.9, 0.2]) ## geel
mesh_r3.paint_uniform_color([0.9, 0.9, 0.9])

# o3d.visualization.draw_geometries([mesh_r1], mesh_show_wireframe=True, mesh_show_back_face=True)
o3d.visualization.draw_geometries([mesh_r1, mesh_r2, mesh_r3], mesh_show_wireframe=True, mesh_show_back_face=True)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
